In [191]:
import pandas as pd
import numpy as np
import abydos
#from abydos import distance

In [192]:
def dep_input_output_col(prev_df,now_df,desc):
    new_col = set(now_df.columns) - set(prev_df.columns)
    old_col = set(list(prev_df.columns))
    sort_old_cols = sorted(old_col,key=lambda x:len(x))[::-1]
    sort_new_cols = sorted(new_col,key=lambda x:len(x))[::-1]
    desc = ops["description"]

    import re
    out_cols = []
    in_cols = []
    for x in sort_new_cols:
        sx = r'\b{}\b'.format(x)
        #print(sx,desc)
        xx = re.findall(sx,desc)
        #print(xx)
        for yy in xx:
            desc = desc.replace(yy,"")
        out_cols = out_cols + xx
    for x in sort_old_cols:
        sx = r'\b{}\b'.format(x)
        #print(sx,desc)
        xx = re.findall(sx,desc)
        #print(xx)
        for yy in xx:
            desc = desc.replace(yy,"")
        in_cols = in_cols + xx
    
    out_cols = list(set(out_cols+list(new_col)))
        
    return in_cols,out_cols
    #ops["description"]

#dep_input_output_col(prev_df,now_df,ops["description"])

# General Instruction

- Cleanup dataset based on the information that is given:
You need to clean the dataset according to the information that is given to you. This means that there are problems with the dataset that need to be fixed, and you should use the information given to you to determine what those problems are and how to fix them.

- Each case has different data quality problems, there will be hint and additional information that can help you understand the problem:
Each row in the dataset may have different data quality problems. There will be hints and additional information provided to help you understand what the specific problem is with each row.

- You can do any approach on cleaning the data, but you should clean the instructed column only:
You have the freedom to use any approach to clean the data, but you should only clean the instructed column. This means that you should not modify any other columns in the dataset, or add or remove any rows.

- Do not create new column or remove any column. Also do not create new row, or remove any row:-
You are not allowed to create new columns or remove any columns from the dataset. You are also not allowed to add or remove any rows.

- Each column will have a flag column something equivalent to <column\_name>\_flag. This column can be used to flag the row if you want to not include it to the downstream task. 0: safe_flag, 1: delete_flag, 2: null_flag (if you want to still include the row with null treatment). You can also add a new category but please add justification and explanation of the new category, there are three categories you can use:
safe_flag (0): this row is safe to use in downstream tasks
delete_flag (1): this row should be deleted and not used in downstream tasks
null_flag (2): this row can be included in downstream tasks but with null treatment.
You can also add a new category, but you need to provide a justification and an explanation for the new category. It is worth to note that the completeness of the dataset is also matter, so try not to flag to many things, and do your best to clean the values.

- For each data cleaning task, we have provided a function that represents the goal of the cleaning. For example, clean_duplicate_id(df) is the function for removing duplicate ID values. These functions take a DataFrame as input and return the cleaned version of the DataFrame.

    In each chunk of data cleaning task, you will see the following three parts:

    1. The clean_<name> function that performs the specific cleaning task.
    2. The execution of the cleaning function on the DataFrame.
    3. A checking part to help you evaluate the effectiveness of the cleaning.
    
  While you can create new cells and add additional code, the cleaning must be performed through the provided cleaning functions. You can adjust the order of the cleaning steps, but please try to move the whole chunks of code to avoid any errors.

The cleaning task will be considered complete if this notebook can be run sequentially by executing "restart and runall"




# Purpose
The purpose of this dataset is to conduct exploratory analysis of the listings and create a prediction model for listing price using some columns from the dataset. This means that the dataset is intended to be used to explore the characteristics and features of the listings, and to build a model that can predict the price of a listing based on certain variables in the dataset. The goal is to gain insights into the factors that influence the price of a listing and to develop a model that can accurately predict listing prices based on those factors.

# Columns and Dataset Description
- id: a unique identifier for each listing.
- name: the name or title of the listing, as provided by the host.
- host_id: a unique identifier for each host.
- host_name: the name of the host who listed the property.
- neighbourhood_group: the larger geographic area in which the listing is located (e.g. a borough or group of neighborhoods).
- neighbourhood: the specific neighborhood in which the listing is located.
- latitude: the latitude coordinate of the listing.
- longitude: the longitude coordinate of the listing.
- room_type: the type of space that is being listed (e.g. an entire apartment, a private room, a shared room).
- price: the nightly price of the listing, in the currency specified in the dataset.
- minimum_nights: the minimum number of nights that a guest must book the listing for.
- number_of_reviews: the total number of reviews that the listing has received.
- last_review: the date of the most recent review of the listing.
- reviews_per_month: the average number of reviews per month that the listing has received.
- calculated_host_listings_count: the total number of listings that the host has on Airbnb.
- availability_365: the number of days per year that the listing is available for booking.
- number_of_reviews_ltm: the total number of reviews that the listing has received in the last 12 months.
- license: a license number for the listing, if applicable (this column may not be present in all versions of the dataset).

Besides the columns above, there are columns pre-defined for flagging the rows based on particular data cleaning context:
- id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the id column (duplicate).
- host_id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the host_id column.
- neighbourhood_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the neighbourhood column.
- latitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the latitude column.
- longitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the longitude column.
- minimum_nights_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the minimum_nights column.
- number_of_reviews_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the number_of_reviews column.
- last_review_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the last_review column.
- room_type_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the room_type column.

In [193]:
# transform dcm
import logging 
import pandas as pd

logging.basicConfig(level=logging.DEBUG)
#logging.basicConfig(level=logging.INFO)

class TransformDCM():
    def __init__(self,trace):
        self.trace = trace
        self.source = []
        self.dataset = []
        self.state = []
        self.array = []
        self.column = []
        self.row = []
        self.cell = []
        self.cell_values = []
        self.column_position = []
        self.row_position = []
        self.user = {None: -1}
        
        self.value_derived_from = []        
        self.col_derived_from = []
        self.state_derived_from = []
        self.col_dependency = []
        self.state_detail = []
        
        self.pd_index = None
        
        self.source_id = 0
        self.dataset_id = 0
        self.array_id = 0
        self.col_id = 0
        self.row_id = 0
        self.cell_id = 0
        self.value_id = 0
        self.state_id = -1
        self.col_pos_id = 0
        self.row_pos_id = 0
        self.user_id = 0
        self.execution_id = 0
        self.prev_state_id = -2
        
        self.col_names_coll = set()
        
        
        self.curr_df = None
        self.curr_col = None
        self.curr_row = None
        self.curr_index = None
        
        self.curr_row_pos = {}
        self.curr_col_pos = {}
        self.curr_col_schema = []
        self.curr_row_list = []
        
        self.curr_state = 0            
        
    
    def render_curr_df(self):
        pass
    
    def render_col(self):
        pass
    
    def render_row(self):
        pass
        
    
    def init_df(self,df):
        self.pd_index = pd.DataFrame(np.empty(df.shape),dtype=object)    
        self.col_names = df.columns
        
        for i,x in enumerate(df.to_records()):
            jj = 0
            #print(i)
            for j,y in enumerate(x):
                #print(j)
                if j==0:
                    continue
                
                self.cell.append((self.cell_id,jj,self.row_id))
                #print(y)
                #self.cell_values.append((self.value_id,self.cell_id,self.state_id,y[0],-1))
                self.cell_values.append((self.value_id,self.cell_id,self.state_id,y,-1))
                #print(self.pd_index.loc[i,jj])
                self.pd_index.loc[i,jj] = (self.cell_id,self.value_id,i,jj) 
                
                self.value_derived_from.append((self.cell_id,self.state_id,-1))

                self.cell_id+=1
                self.value_id+=1
                if i == 0:
                    if jj == 0:
                        prev_j = -1
                    
                    
                    self.column.append((self.col_id,self.array_id))                    
                    self.column_position.append((self.col_pos_id,self.col_id,self.state_id,self.col_names[jj],prev_j,-1))
                    self.col_names_coll.add(self.col_names[jj])
                    self.curr_col_schema.append((self.col_names[jj],jj,self.col_id,prev_j))
                    self.curr_col_pos[self.col_id] = (self.col_pos_id,prev_j)
                    prev_j = jj
                    self.col_pos_id+=1                    
                    self.col_id+=1    
                jj+=1
            if i == 0:
                prev_i = -1
                
            self.row.append((self.row_id,self.array_id))            
            self.row_position.append((self.row_pos_id,self.row_id,self.state_id,prev_i,-1))
            self.curr_row_pos[self.row_id] = (self.row_pos_id,prev_i)
            self.curr_row_list.append(self.row_pos_id)
            prev_i = self.row_id
            self.row_id+=1       
            self.row_pos_id+=1            
        
        """            
        col_id = np.where(df.columns==col.name)[0][0]
        columns.append((col_id,self.array_id))
        #temp_col = []
        for i,x in enumerate(col):
            if not row_processed:
                rows.append((i,array_id))
            #temp_col.append((cell_id,col_id,i))
            cells.append((cell_id,col_id,i))
            cell_values.append((value_id,cell_id,state_id,x,-1))
            pd_index.loc[i,col_id] = (cell_id,value_id,col_id,i)        
            cell_id+=1
            value_id+=1
            #print(i,col_id)
        row_processed = True        
        """
    
    def init_dataset(self,tt):
        logging.debug("init dataset")
        # get filename from trace        
        df = tt[5]
        code = tt[6]
        self.source.append((self.source_id,code,"dataframe"))
        self.dataset.append((self.dataset_id,self.source_id))
        self.array.append((self.array_id,self.dataset_id))
        # state creation
        
        prev_state_id = self.state_id
        self.state_id+=1        
        self.state.append((self.state_id,prev_state_id))            
        
        # generate column, row, cell
        self.init_df(df)
        self.state_id+=1        

        
        #self.state_id+=1
        #self.array_id+=1
        #self.dataset_id+=1
        #self.source_id+=1
        
    def init_dataset_df(self,df,state_ss={'op':"initial"},fname=None):
        logging.debug("init dataset")
        self.source.append((self.source_id,fname,"dataframe"))
        self.dataset.append((self.dataset_id,self.source_id))
        self.array.append((self.array_id,self.dataset_id))
        # state creation
                
        # generate column, row, cell
        self.init_df(df)       

        prev_state_id = self.state_id
        self.state_id+=1
        self.state.append((self.state_id,prev_state_id))
        self.state_detail.append((prev_state_id,state_ss))
    
    def change_column_schema(self,prev_df,now_df):
        #old_col = list(self.curr_col)  
        old_col = list(prev_df.columns)
        now_col = list(now_df.columns)
        #logging.debug(now_df)
        #logging.debug(("old new col",old_col,self.curr_col_schema,now_col))
        new_col = set(now_col)-set(old_col)
        ocol = set(old_col) - set(now_col)
        
        logging.debug(("change_col_schema old_col",old_col,self.curr_col_schema,now_col))
        logging.debug(("change_col_schema new_col",new_col,ocol))
        
        old_schema = [x[0] for x in self.curr_col_schema]
        
        temp_new_col = []
        
        temp_prev = None
        
        logging.debug(("change_col_schema old_schema:",old_schema))
                
        for n_idx,x in enumerate(now_col):
            logging.debug("change_col_schema now_col:")
            try:
                idx_schema = old_schema.index(x)
            except:
                # check the potential column
                for y in ocol:                    
                    test = self.curr_df.loc[:,y].fillna(0) == now_df.loc[:,x].fillna(0)
                    logging.debug(("test",test.sum(),self.curr_df.shape[0],self.curr_df,now_df))
                    if test.sum() == self.curr_df.shape[0]:
                        idx = old_col.index(y)
                        prev_idx = (-1,None)
                        logging.debug(("idx_test",idx))
                        if idx > 0:
                            prev_idx = old_col[idx-1]
                            idx_n = now_col.index(x)
                            now_idx = self.curr_col_schema[idx_n]
                            prev_col_schema = (None,None,-1,None)
                            if idx_n > 0:
                                prev_col_schema = self.curr_col_schema[idx_n-1]
                            logging.debug(("change_col_schema now_idx:",now_idx,prev_idx))
                            self.column_position.append((self.col_pos_id,now_idx[-2],self.state_id,x,prev_col_schema[2],now_idx[1]))
                            logging.debug(("change_col_schema adding_col_pos1:",(self.col_pos_id,now_idx[-2],self.state_id,x,n_idx-1,now_idx[2])))
                            temp_new_col.append((x,self.col_pos_id,now_idx[2],now_idx[3]))
                            #temp_new_col.append((x,self.col_pos_id,now_idx[2]))
                            #temp_new_col.append((x,self.col_pos_id,-1))
                            self.col_pos_id+=1  
                            
                            logging.debug(("change_col_schema now_idx",now_idx))
                            self.col_dependency.append((self.state_id,now_idx[-2] if len(now_idx)==4 else now_idx[-1],now_idx[-2] if len(now_idx)==4 else now_idx[-1]))                                                                      
                            break
                continue
                                        
            old_schema_idx = self.curr_col_schema[idx_schema]
            idx = idx_schema
            #idx = old_col.index(x)
            #prev_idx = (-1,None)
            prev_idx = -1
            prev_old_schema = (None,None,None,None)
            if idx > 0:
                prev_idx = idx - 1
                prev_old_schema = self.curr_col_schema[prev_idx]            

            prev_nidx = -1
            prev_new_schema = (None,None,None,None)
            if n_idx > 0:     
                prev_nidx = n_idx - 1
                prev_new_schema = temp_new_col[-1]
                
            #logging.debug(("change_col_schema old",n_idx,x,idx,prev_idx,self.curr_col_schema[idx_schema]))
            #logging.debug(("change_col_schema cur_col_schema:",self.curr_col_schema[n_idx][0],x))
            logging.debug(("change_col_schema prev next:",prev_old_schema,prev_new_schema))
            
            #if self.curr_col_schema[n_idx][0] != x:
            if prev_old_schema[2] != prev_new_schema[2]:
                #if prev_idx[1] != prev_nidx[1]: 
                #if prev_idx[0] != prev_nidx[1]:         
                #logging.debug(("tempnewcol:",temp_new_col[n_idx-1], now_idx))
                #if temp_new_col[n_idx-1][2] != now_idx[2]:
                """
                if temp_new_col[n_idx-1][2] != now_idx[2]:
                    idx_n = now_col.index(x)
                    self.column_position.append((self.col_pos_id,now_idx[-2],self.state_id,x,temp_new_col[n_idx-1][2],now_idx[2]))
                    logging.debug(("adding_col_pos2:",(self.col_pos_id,now_idx[-2],self.state_id,x,temp_new_col[n_idx-1][2],now_idx[2])))                            
                    #temp_new_col.append((x,self.col_pos_id,now_idx[1],now_idx[2],temp_prev))
                    temp_new_col.append((x,self.col_pos_id,now_idx[2],temp_prev))
                    self.col_pos_id+=1
                """
                self.column_position.append((self.col_pos_id,old_schema_idx[1],self.state_id,x,prev_new_schema[1],old_schema_idx[1]))
                #logging.debug(("adding_col_pos2:",(self.col_pos_id,now_idx[-2],self.state_id,x,temp_new_col[n_idx-1][2],now_idx[2])))                            
                #temp_new_col.append((x,self.col_pos_id,now_idx[1],now_idx[2],temp_prev))
                temp_new_col.append((x,self.col_pos_id,old_schema_idx[1],prev_new_schema[1]))
                self.col_pos_id+=1
                #aaa
            else:
                 #temp_new_col.append((x,now_idx[1],now_idx[2],now_idx[3]))
                temp_new_col.append(self.curr_col_schema[idx_schema])
        
        
        logging.debug(("change_col_schema temp_new_col:",temp_new_col))
        
        self.curr_col_schema = temp_new_col
                
        return None
        
        for n_idx,x in enumerate(now_col):
            try:
                idx_schema = old_schema.index(x)
            except:
                # check the potential column
                for y in ocol:                    
                    test = self.curr_df.loc[:,y].fillna(0) == now_df.loc[:,x].fillna(0)
                    logging.debug(("test",test.sum(),self.curr_df.shape[0],self.curr_df,now_df))
                    if test.sum() == self.curr_df.shape[0]:
                        idx = old_col.index(y)
                        prev_idx = (-1,None)
                        logging.debug(("idx_test",idx))
                        if idx > 0:
                            prev_idx = old_col[idx-1]
                            idx_n = now_col.index(x)
                            now_idx = self.curr_col_schema[idx_n]  
                            logging.debug(("now_idx:",now_idx,prev_idx))
                            self.column_position.append((self.col_pos_id,now_idx[-2],self.state_id,x,n_idx-1,now_idx[2]))
                            logging.debug(("adding_col_pos1:",(self.col_pos_id,now_idx[-2],self.state_id,x,n_idx-1,now_idx[2])))
                            temp_new_col.append((x,self.col_pos_id,now_idx[2],now_idx[3]))
                            #temp_new_col.append((x,self.col_pos_id,now_idx[2]))
                            #temp_new_col.append((x,self.col_pos_id,-1))
                            self.col_pos_id+=1  
                            
                            logging.debug(("now_idx",now_idx))
                            self.col_dependency.append((self.state_id,now_idx[-2] if len(now_idx)==4 else now_idx[-1],now_idx[-2] if len(now_idx)==4 else now_idx[-1]))                              
                continue
            
            old_schema_idx = self.curr_col_schema[idx_schema]
            
            idx = old_col.index(x)
            prev_idx = (-1,None)
            next_idx = None
            if idx > 0:
                prev_idx = old_col[idx-1]
            #if idx < len(old_col)-1:
            #    next_idx = old_col[idx+1]

            logging.debug(("change_schema",n_idx,x,idx,prev_idx,self.curr_col_schema[idx_schema]))
                        
            now_idx = self.curr_col_schema[idx_schema]    
            
            prev_nidx = (-1,None)
            if n_idx > 0:                
                prev_nidx = (n_idx-1,now_col[n_idx-1])                        
                prev_idx = self.curr_col_schema[n_idx-1]
            
            #logging.debug((self.curr_col_pos[old_schema_idx[2]],now_idx))
            
            
            if temp_prev == None:
                temp_prev = -1
            else:
                temp_prev = now_idx[-1]
                
            #if self.curr_col_pos[old_schema_idx[2]][1] != now_idx[3]:
            #print(now_idx,temp_new_col[n_idx-1])
            logging.debug(("prev_idx,nidx:",prev_idx,prev_nidx,temp_new_col,n_idx))
            if prev_idx[1] != prev_nidx[1]:         
                #if prev_idx[0] != prev_nidx[1]:         
                logging.debug(("tempnewcol:",temp_new_col[n_idx-1], now_idx))
                #if temp_new_col[n_idx-1][2] != now_idx[2]:
                if temp_new_col[n_idx-1][2] != now_idx[2]:
                    idx_n = now_col.index(x)
                    self.column_position.append((self.col_pos_id,now_idx[-2],self.state_id,x,temp_new_col[n_idx-1][2],now_idx[2]))
                    logging.debug(("adding_col_pos2:",(self.col_pos_id,now_idx[-2],self.state_id,x,temp_new_col[n_idx-1][2],now_idx[2])))                            
                    #temp_new_col.append((x,self.col_pos_id,now_idx[1],now_idx[2],temp_prev))
                    temp_new_col.append((x,self.col_pos_id,now_idx[2],temp_prev))
                    self.col_pos_id+=1
            else:
                 temp_new_col.append((x,now_idx[1],now_idx[2],now_idx[3]))
            
                                
            """
            #if idx_n > 0:
            prev_idx_n = -1
            try:
                prev_idx_n = now_col[idx_n-1]
            except:
                pass
            logging.debug(("test column",prev_idx,prev_idx_n))

            if prev_idx is not None and prev_idx[0]!=prev_idx_n:
                self.column_position.append((self.col_pos_id,prev_idx[0],self.state_id,x,prev_idx_n,prev_idx[1]))
                self.col_pos_id+=1        
            #self.curr_col_schema.insert(idx,(x,self.col_id))            
            self.col_names_coll.add(x)
            """
            
        self.curr_col_schema = temp_new_col
            
        pass
    
    def change_row_position(self,prev_df,now_df):
        old_row = list(self.curr_row)
        now_row = list(now_df.index)
        #new_row = set(now_row)-set(old_row)
        
        temp_cur_row_pos = []
        for x in now_row:
            #logging.debug((x))
            temp_cur_row_pos.append(self.curr_row_pos[x])            
        
        logging.debug(temp_cur_row_pos)
        pass
    
    def add_column(self,prev_df,df,state_ss=None):   
        #old_col = self.curr_col
        old_col = prev_df.columns
        now_col = df.columns
        new_col = set(now_col)-set(old_col)
        logging.debug(("new_col:",new_col,self.curr_col_schema))
        
        # sort new_col by the index
        new_col = list(filter(lambda x:x in new_col,now_col))

        in_col,out_col = dep_input_output_col(prev_df,df,state_ss["operation"]["description"])
        #print("in_col,out_col",in_col,out_col)
        
        old_col_schema = self.curr_col_schema.copy()
        
        for j,x in enumerate(new_col):
            last_col = self.curr_col_schema[-1][1]
            new_col_val = df.loc[:,[x]]
            logging.debug(new_col_val.values.tolist())
            idx = list(now_col).index(x)
                
            logging.debug(idx)
            if idx > 0:
                prev_idx = self.curr_col_schema[idx-1]
            
            self.column_position.append((self.col_pos_id,self.col_id,self.state_id,x,prev_idx[2],-1))       
            logging.debug(("curr_col_schema",self.curr_col_schema))
            #self.curr_col_schema.insert(idx,(x,self.col_pos_id,self.col_id,self.curr_col_schema[-1][3] if len(self.curr_col_schema[-1])==4 else self.curr_col_schema[-1][-1]))
            self.curr_col_schema.insert(idx,(x,self.col_pos_id,self.col_id,self.curr_col_schema[-1][2]))
            self.curr_col_pos[self.col_id] = (self.col_pos_id,last_col)
            self.col_names_coll.add(x)            
            
            # add values
            self.column.append((self.col_id,self.array_id))
            #self.cell_values.append()
            temp_idx = []
            for y,i in zip(new_col_val.values.tolist(),self.curr_row):
            #for y,i in zip(new_col_val.values.tolist(),self.curr_row):
                self.cell.append((self.cell_id,self.col_id,i))
                self.cell_values.append((self.value_id,self.cell_id,self.state_id,y[0][0] if y[0]!=None else y[0],-1))
                #print(self.pd_index.loc[i,jj])
                #self.pd_index.loc[i,jj] = (self.cell_id,self.value_id,i,jj) 
                temp_idx.append([(self.cell_id,self.value_id,i,self.col_id)])
                
                # add linkage to the derived by cell
                # lookup value id for input col_value
                logging.debug(("in_col:",in_col,self.curr_col_schema))
                for z in in_col:                    
                    zzl = [x[-2] if len(x)==4 else x[-1] for x in list(filter(lambda x:(x[0]==z),self.curr_col_schema))]
                    logging.debug(("zzl:",zzl))
                    for zz in zzl:                        
                        op_cid = list(filter(lambda x:(x[1]==zz)&(x[2]==i),self.cell))
                        logging.debug(("op_cid:",op_cid))
                        for zzz in op_cid:
                            op_val = list(filter(lambda x:x[1]==zzz[0],sorted(self.cell_values,key=lambda x:x[0])))[::-1]
                            logging.debug(("op_val:",op_val))
                            for v in op_val:
                                self.value_derived_from.append((self.value_id,self.state_id,v[0]))
                                break
                                            
                #filter(lambda x:,self.cell_values)
                
                self.value_id+=1
                self.cell_id+=1
            
            #self.pd_index[self.col_id] = pd.DataFrame(temp_idx)
            self.pd_index.insert(loc=idx,column=self.col_id,value=pd.DataFrame(temp_idx)[0].tolist())
            
            #self.curr_col_pos[self.col_id] = (self.col_pos_id,prev_j)

            for z in in_col:
                zzl = [x[-2] for x in list(filter(lambda x:(x[0]==z),self.curr_col_schema))]
                for zz in zzl:                        
                    self.col_dependency.append((self.state_id,self.col_id,zz))                                            
            
            #prev_j = j
            self.col_pos_id+=1                    
            self.col_id+=1
        
        # normalize column position
        temp_new_col = []
        for n_idx,x in enumerate(self.curr_col_schema):
            filtered_col = list(filter(lambda y: y[2] == x[2],old_col_schema))
            if len(filtered_col)>0:
                idx = [y[2] for y in old_col_schema].index(x[2])
                prev_idx = -1
                prev_old_schema = (None,None,None,None)
                if idx > 0:
                    prev_idx = idx - 1
                    prev_old_schema = old_col_schema[prev_idx]
                old_schema_idx = old_col_schema[idx]
                
                prev_nidx = -1
                prev_new_schema = (None,None,None,None)
                if n_idx > 0:     
                    prev_nidx = n_idx - 1
                    prev_new_schema = self.curr_col_schema[prev_nidx]

                if prev_old_schema[2] != prev_new_schema[2]:                
                    self.column_position.append((self.col_pos_id,old_schema_idx[1],self.state_id,x[0],prev_new_schema[1],old_schema_idx[1]))
                    
                    temp_new_col.append((x[0],self.col_pos_id,old_schema_idx[1],prev_new_schema[1]))
                    self.col_pos_id+=1                
                else:
                    temp_new_col.append(old_col_schema[idx])
            else:
                temp_new_col.append(self.curr_col_schema[n_idx])
        
        logging.debug(("add column",temp_new_col))
        
        self.curr_col_schema = temp_new_col
                        
        #len(now_col)>len(prev_col)
        pass
    
    def remove_column(self,prev_df,df):
        #old_col = self.curr_col
        old_col = prev_df.columns
        curr_col = self.curr_col_schema
        now_col = df.columns
        
        removed = set(old_col) - set(now_col)
        logging.debug(("remove_column: ",removed))
        for x in removed:
            idx = list(old_col).index(x)

            #logging.debug(idx)
            prev_idx = None
            next_idx = None
            if idx > 0:
                prev_idx = self.curr_col_schema[idx-1]
            if idx < len(old_col)-1:
                next_idx = self.curr_col_schema[idx+1]
            
            self.column_position.append((self.col_pos_id,self.curr_col_schema[idx][1],self.state_id,x,-2,self.curr_col_schema[idx][1]))
            self.curr_col_pos[idx] = (self.col_pos_id,-2)
            old_col = self.curr_col_schema.pop(idx)
            
            
            self.col_pos_id+=1          
            if next_idx is not None:
                self.column_position.append((self.col_pos_id,next_idx[1],self.state_id,x,prev_idx[1],next_idx[0]))                
                self.col_pos_id+=1                                        
        pass
    
    def add_row(self,df):
        old_row = list(self.curr_row)
        now_row = list(df.index)
        new_row = set(now_row)-set(old_row)           
        logging.debug((old_row,now_row,new_row))
        
        
        for x in new_row:
            new_row_val = df.loc[[x],:]
            logging.debug(("new row",new_row_val.values.tolist(),self.curr_col_schema))
        
            for y,i in zip(new_row_val.values.tolist()[0],self.curr_col_schema):
                #print(i)
                self.cell.append((self.cell_id,i[1],self.row_id))
                self.cell_values.append((self.value_id,self.cell_id,self.state_id,y[0],-1))
                self.cell_id+=1
                self.value_id+=1
            
            self.row.append((self.row_id,self.array_id))            
            
            if self.curr_row_pos[self.row_id-1][1]!=-2:
                self.curr_row_pos[self.row_id] = (self.row_pos_id,self.row_id-1)
                self.row_position.append((self.row_pos_id,self.row_id,self.state_id,self.row_id-1,-1))
            else:
                temp_prev_row = list(filter(lambda x:(x[1]==self.row_id-1)&(x[3]!=-2),self.row_position))
                if len(temp_prev_row)>0:
                    self.curr_row_pos[self.row_id] = (self.row_pos_id,temp_prev_row[-1][3])
                    self.row_position.append((self.row_pos_id,self.row_id,temp_prev_row[-1][3],-1))

                else: 
                    self.curr_row_pos[self.row_id] = (self.row_pos_id,self.row_id-1)
            
            self.row_id+=1    
            self.row_pos_id+=1
                
            """
            idx = list(now_col).index(x)
                
            logging.debug(idx)
            if idx > 0:
                prev_idx = self.curr_col_schema[idx-1]
            
            self.column_position.append((self.col_pos_id,self.col_id,self.state_id,x,prev_idx[1],-1))
            
            self.curr_col_schema.insert(idx,(x,self.col_id))
            
            self.col_names_coll.add(x)
            
            
            # add values
            self.column.append((self.col_id,self.array_id))
            #self.cell_values.append()
            temp_idx = []
            for y,i in zip(new_col_val.values.tolist(),self.curr_row):
                self.cell.append((self.cell_id,self.col_id,i))
                self.cell_values.append((self.value_id,self.cell_id,self.state_id,y[0],-1))
                #print(self.pd_index.loc[i,jj])
                #self.pd_index.loc[i,jj] = (self.cell_id,self.value_id,i,jj) 
                temp_idx.append([(self.cell_id,self.value_id,i,self.col_id)])
                self.value_id+=1
                self.cell_id+=1
            
            self.pd_index[self.col_id] = pd.DataFrame(temp_idx)

            #prev_j = j
            self.col_pos_id+=1                    
            self.col_id+=1             
            """
        
        
        pass

    def remove_row(self,df):
        old_row = list(self.curr_row)
        now_row = list(df.index)
        removed_row = set(old_row)-set(now_row)           
        logging.debug((old_row,now_row,removed_row))
        
        
        for x in removed_row:
            temp_row_pos = self.curr_row_pos[x]

            self.row_position.append((self.row_pos_id,x,self.state_id,-2,temp_row_pos[0]))
            self.curr_row_pos[x] = (self.row_pos_id,-2)
            self.row_pos_id+=1
            # next pos

            # filter with previous row_pos_id
            next_row = list(filter(lambda x:x[1][1]==x,self.curr_row_pos.items()))
            if len(next_row)>0:
                next_row = next_row[0]
                self.row_position.append((self.row_pos_id,next_row[0],self.state_id,temp_row_pos[1],next_row[1][0]))
                self.curr_row_pos[next_row[0]] = (self.row_pos_id,temp_row_pos[1])

                self.row_pos_id+=1        
        pass
    
    def change_values(self,df,change):
        #self.curr_index = self.pd_index[change]
        
        #change.columns = df.columns
        #change.index = df.index                
        
        #l_idx = self.pd_index[change]
                
        tt = np.where(np.matrix(change.to_numpy())==True)
        #print(tt)
        tt = list(zip(tt[0],tt[1]))
        #logging.debug((list(zip(tt[0],tt[1]))))
        #if tt[0].shape[0] == 1:
        #    tt = [(tt[0][0],tt[1][0])]
        #logging.debug((np.where(np.matrix(change.to_numpy())==True)))
        #logging.debug(("tt",tt))
        logging.debug(("change_values pd_index:",self.pd_index,change.to_numpy()))
        idx = self.pd_index.to_numpy()[change.to_numpy()].flatten()        
        val = df[change].to_numpy().flatten()
        logging.debug(("idx:",idx,val))
        #list(idx)
        idx_list = list(filter(lambda x:pd.isna(x)!=True,idx))
        val_list = list(filter(lambda x:pd.isna(x)!=True,val))
        
        logging.debug(("change_values idxlist:",idx_list,val_list))
        
        #?filter
        
        #print(idx_list,val_list)
        
        set_temp_col = set()

        for x in zip(idx_list,val_list,tt):
            logging.debug(("change_values idxlist:",x))
            self.cell_values.append((self.value_id,x[0][0],self.state_id,x[1],x[0][1]))   
            #logging.debug(("x2",x[2]))
            ttx = list(self.pd_index.loc[x[2][0],x[2][1]])
            ttx[1] = self.value_id
            #logging.debug((ttx))
            self.pd_index.loc[x[2][0],x[2][1]] = tuple(ttx)
            #x[0] = self.value_id            
            
            
            self.value_derived_from.append((self.value_id,self.state_id,x[0][1]))
                        
            temp_col = list(filter(lambda y: y[0]==x[0][0],self.cell))[0][1]   
            if temp_col not in set_temp_col:
                self.col_dependency.append((self.state_id,temp_col,temp_col))  
                set_temp_col.add(temp_col)
                #aaa
            
            """            
            logging.debug(("in_col",in_col))
            for z in in_col:
                zzl = [x[-1] for x in list(filter(lambda x:(x[0]==z),self.curr_col_schema))]
                logging.debug(("zzl",zzl))
                for zz in zzl:                        
                    op_cid = list(filter(lambda x:(x[1]==zz)&(x[2]==i),self.cell))
                    logging.debug(("op_cid",op_cid))
                    for zzz in op_cid:
                        op_val = list(filter(lambda x:x[1]==zzz[0],self.cell_values))
                        logging.debug(("op_val",op_val))
                        for v in op_val:
                            self.value_derived_from.append((self.value_id,v[0]))
            """
            
            self.value_id+=1
            #print(x)

        #prev_state_id = state_id
        #state_id+=1
        #state.append((state_id,prev_state_id))
        
        pass
                    
    
    def change_df(self,prev_df,now_df,state_ss,user=None,prev_ss=None):  
        try:
            in_col,out_col = dep_input_output_col(prev_df,now_df,state_ss["operation"]["description"])
        
            # add linkage to the derived by cell
            # lookup value id for input col_value
            if len(out_col)==0:
                logging.debug(("in_col:",in_col,self.curr_col_schema))
                for z in in_col:
                    zzl = [x[-2] if len(x)==4 else x[-1] for x in list(filter(lambda x:(x[0]==z),self.curr_col_schema))]
                    logging.debug(("zzl",zzl))
                    for zzy in zzl:
                        self.col_dependency.append((self.state_id,zzy,zzy))                                
        except:
            pass
        
        #prev_col = np.array(self.curr_col)
        prev_col = prev_df.columns
        now_col = now_df.columns
        
        #print(prev_col,now_col)
        
        
        # condition for add and remove columns
        if len(now_col)>len(prev_col):
            # condition for add_columns:
            logging.debug("add column")
            self.add_column(prev_df,now_df,state_ss)            
        elif len(prev_col)>len(now_col):
            # condition for remove_columns:
            logging.debug("remove_column")
            self.remove_column(prev_df,now_df)
            
        # condition for change of schema:
        if len(prev_col)==len(now_col) and np.sum(prev_col!=now_col)>0:
            logging.debug("change column schema")
            self.change_column_schema(prev_df,now_df)
        
        #prev_row = np.array(self.curr_row)
        prev_row = np.array(list(prev_df.index))
        now_row = np.array(list(now_df.index))
        
        # condition for add and remove rows        
        if len(now_row)>len(prev_row):
            # condition for add_columns:
            logging.debug("add row")
            self.add_row(now_df)            
        elif len(prev_row)>len(now_row):
            # condition for remove_columns:
            logging.debug("remove row")
            self.remove_row(now_df)
            
        # condition for change of schema:
        if len(prev_row)==len(now_row) and np.sum(prev_row!=now_row)>1:
            logging.debug("change row position")
            self.change_row_position(prev_df,now_df)
            
        
        # condition for change of values
        try:
            change_val = now_df.fillna("null")!=prev_df.fillna("null")
            if change_val.to_numpy().sum()>0:
                logging.debug("change values")
                self.change_values(now_df,change_val)
            else:
                logging.debug("nothing change")
        except BaseException as ex:
            logging.debug(ex)
            pass
                         
        #prev_state_id = self.state_id
        #if self.prev_state_id
        try:
            self.user[user]
        except:
            self.user[user]=self.user_id
            self.user_id+=1
        
        if prev_ss == None:
            self.state_detail.append((self.state_id,state_ss,self.execution_id,self.prev_state_id,self.user[user]))       
            self.state.append((self.state_id,self.prev_state_id))    
            self.prev_state_id = self.state_id
            self.state_id+=1   
        else:             
            self.state_detail.append((self.state_id,state_ss,self.execution_id,prev_ss,self.user[user]))       
            self.state.append((self.state_id,prev_ss))       
            self.prev_state_id = self.state_id
            self.state_id+=1
            
        self.execution_id+=1       
        
        return True
        
        """
        
        col_id = np.where(prev_df.columns==col.name)[0][0]
        columns.append((col_id,array_id))
        #temp_col = []
        for i,x in enumerate(col):
            if not row_processed:
                rows.append((i,array_id))
            #temp_col.append((cell_id,col_id,i))
            cells.append((cell_id,col_id,i))
            cell_values.append((value_id,cell_id,state_id,x,-1))
            cell_id+=1
            value_id+=1
            #print(i,col_id)
            pd_index.loc[i,col_id] = (cell_id,col_id,i)            
        row_processed = True
        state_id+=1
        """
        
    def transform(self):
        prev_df = None
        now_df = None
        for i,x in enumerate(self.trace):
            now_df = x[5]
            #print(now_df)
            if i == 0:
                #init dataset
                self.init_dataset(x)
            else:
                self.change_df(prev_df,now_df)
                pass
            
            
            self.curr_col = now_df.columns
            #self.curr_col_schema = [for x in self.curr_col]
            self.curr_row = list(now_df.index)        
            self.curr_df = now_df                        
            prev_df = now_df
        
        

# Load Data

In [194]:
original_pd = pd.read_csv("../case_1/chicago_vert_dataset.csv",dtype=str).append(pd.read_csv("../case_2/chicago_vert_dataset.csv",dtype=str))
original_pd = original_pd.reset_index(drop=True)
for x in list(filter(lambda x:x.endswith("flag"),original_pd.columns)):
    original_pd.loc[:,x] = original_pd.loc[:,x].astype(int) 
original_pd_before = original_pd.copy()

/tmp/ipykernel_3425/3899981369.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  original_pd = pd.read_csv("../case_1/chicago_vert_dataset.csv",dtype=str).append(pd.read_csv("../case_2/chicago_vert_dataset.csv",dtype=str))
/tmp/ipykernel_3425/3899981369.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  original_pd.loc[:,x] = original_pd.loc[:,x].astype(int)


In [195]:
group1_index = pd.read_csv("../case_1/chicago_vert_dataset.csv",dtype=str).index
group1_index
group2_index = pd.Index(range(group1_index.stop,original_pd.shape[0]))

In [196]:
group1_index,group2_index

(RangeIndex(start=0, stop=1486, step=1),
 RangeIndex(start=1486, stop=3201, step=1))

In [197]:
original_pd.tail(50)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
3151,764622652958240880,Group book 3 Penthouse by Cloud9,248760412,Cloud9,NaN,Near West Side,4188764,-87.64578,Entire home/apt,1488,...,R22000092079,0,0,0,0,0,0,0,0,0
3152,764669447707306735,South Loop Chicago-5 min to DT,448589644,Andrea,NaN,Near South Side,41.86006,-87.62485,Entire home/apt,120,...,NaN,0,0,0,0,0,0,0,0,0
3153,765003851418902674,"Bucktown 1br w/ roof & bbq, nr blue line",107434423,Blueground,NaN,WeDstYC Town,41.91152789999999,-87.67875529999998,Entire home/apt,145,...,NaN,0,0,0,0,0,0,0,0,0
3154,765213311537655730,Classic Hp 1 Br with Fast Transit to UChicago Dt,47172572,Zencity,NaN,KgenRwoood,41.80302,-87.59742,Apartment,70,...,R22000092864,0,0,0,0,0,0,0,0,0
3155,765213311537655730,Classic Hp 1 Br with Fast Transit to UChicago Dt,47172572,Zencity,NaN,KgenRwoood,41.80302,-87.59742,Apartment,70,...,R22000092864,0,0,0,0,0,0,0,0,0
3156,766069261296774780,Old-World Unit 3 Downtown!,170785489,Dmd,NaN,Near North Side,41.90117,-87.6257,Entire home/apt,137,...,2210122,0,0,0,0,0,0,0,0,0
3157,766499608228095208,"Clean modern apt w/ parking, laundry",36989519,Mushky,NaN,West Ridge,41.99345,-87.70609,Entire home/apt,91,...,R22000091827,0,0,0,0,0,0,0,0,0
3158,766657991814572268,Smoke n Stay - 2BR 10min from DT,468900645,Dana,NaN,SoutHhtk Chicago,41.75017,-87.55126,Entire home/apt,99,...,R22000088337,0,0,0,0,0,0,0,0,0
3159,767226985954458283,"W. Loop 2br w/ gym & coworking, nr UIC",107434423,Blueground,NaN,Near West Side,41.8763,-87.65371999999999,Entire home/apt,118,...,NaN,0,0,0,0,0,0,0,0,0
3160,767434780399355268,100 home away from home,70728663,Shelby,NaN,Bridgeport,41.82881576674946,-87.64687367897614,Private room,68,...,NaN,0,0,0,0,0,0,0,0,0


In [198]:
super_ori_pd = pd.read_csv("../../../collaboration_simulation/airbnb_test_case/chicago_listings.csv",dtype=str)
for x in list(filter(lambda x:x.endswith("flag"),original_pd.columns)):
    super_ori_pd.loc[:,x] = 0
#super_ori_pd = super_ori_pd.loc[super_ori_pd.id.isin(original_pd.id),original_pd.columns]
#super_ori_pd
super_ori_pd = original_pd.merge(super_ori_pd,left_on="id",right_on="id",suffixes=("_x",""))
super_ori_pd = super_ori_pd.loc[:,original_pd.columns].reset_index(drop=True)

In [199]:
original_pd.describe()

,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
count,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [200]:
super_ori_pd.describe()

,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
count,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0,3201.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [201]:
original_pd

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,25879,2/1 One Block to Fullerton L Red Line Deck & ...,101521,Red,NaN,Lincoln Park,41.92693,-87.65753,Entire home/apt,94,...,City registration pending,0,0,0,0,0,0,0,0,0
1,37738,Andersonville - Perfect location!,162364,Mat And Randy,NaN,Uptown,41.9729,-87.66538,Private room,110,...,R20000059426,0,0,0,0,0,0,0,0,0
2,189821,"Best in Chicago, private, amazing garden space",899757,Meighan,NaN,Logan Square,41.92918,-87.70219,Entire home/apt,236,...,R21000062936,0,0,0,0,0,0,0,0,0
3,207218,Historic Pullman Artist Flat - Artists & Explo...,1019125,Jb,NaN,Pullman,41.6883,-87.60892,Entire home,100,...,R21000073121,0,0,0,0,0,0,0,0,0
4,220333,Pullman School House Apartment - monthly rental,1019125,Jb,NaN,Pullman,41.68815,-87.60918,Entire home/apt,100,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,782628636832878491,"Steps to Shop, Eat, Train | Easy Access | Zencity",47172572,Zencity,NaN,West Town,41.8955,-87.66124,Entire home/apt,50,...,R19000043484,0,0,0,0,0,0,0,0,0
3197,782643895516370805,Old Town Oasis,169297663,William,NaN,Near North Side,41.90105,-87.63716,Apartment,120,...,R22000093645,0,0,0,0,0,0,0,0,0
3198,784994899201350568,Lovely 1 bed Apt in River North,52827024,Yakir,NaN,Near West Side,41.88822006416301,-87.64145321718578,Entire home/apt,84,...,NaN,0,0,0,0,0,0,0,0,0
3199,785423932330914663,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,NaN,Near North Side,41.890516,-87.635955,Apartment,169,...,NaN,0,0,0,0,0,0,0,0,0


In [202]:
original_pd.shape,super_ori_pd.shape

((3201, 27), (3201, 27))

In [203]:
import numpy as np
import json
import pandas as pd
import pickle

#dcm1 = TransformDCM(None)
with open("dcm_collab.pickle","rb") as file:
    dcm1 = pickle.load(file)

In [204]:
#dcm1.init_dataset_df(super_ori_pd)

In [205]:
#dcm1.curr_col = super_ori_pd.columns
#dcm1.curr_row = list(super_ori_pd.index) 
#dcm1.curr_df = super_ori_pd

In [206]:
#dcm1.change_df(super_ori_pd,original_pd,"perturbed_dataset","s1")

In [207]:
xx = pd.DataFrame(dcm1.cell_values)
xx[xx[2]==0].merge(xx,left_on=4,right_on=0)

,4,0_x,1_x,2_x,3_x,4_x,0_y,1_y,2_y,3_y,4_y
0,66,86427,66,0,"November 28, 2022",66,66,66,-1,2022-11-28,-1
1,89,86428,89,0,Entire home,89,89,89,-1,Entire home/apt,-1
2,167,86429,167,0,Lhke eieu,167,167,167,-1,Lake View,-1
3,170,86430,170,0,Entire home,170,170,170,-1,Entire home/apt,-1
4,224,86431,224,0,Entire home,224,224,224,-1,Entire home/apt,-1
...,...,...,...,...,...,...,...,...,...,...,...
2286,86246,88713,86246,0,Apartment,86246,86246,86246,-1,Entire home/apt,-1
2287,86273,88714,86273,0,Apartment,86273,86273,86273,-1,Entire home/apt,-1
2288,86327,88715,86327,0,Apartment,86327,86327,86327,-1,Entire home/apt,-1
2289,86381,88716,86381,0,Apartment,86381,86381,86381,-1,Entire home/apt,-1


In [208]:
dcm1.state_detail

[(-1, {'op': 'initial'}, 0, -2, None),
 (0, 'perturbed_dataset', 0, -1, 0),
 (1, 'clean_duplicate_id_c1', 1, 0, 1),
 (2, 'clean_inconsistent_host_id_c1', 2, 1, 1),
 (3, 'clean_neighbourhood_c1', 3, 2, 1),
 (4, 'clean_latitude_c1', 4, 3, 1),
 (5, 'clean_longitude_c1', 5, 4, 1),
 (6, 'clean_room_type_c1', 6, 5, 1),
 (7, 'clean_minimum_nights_c1', 7, 6, 1),
 (8, 'clean_number_of_reviews_c1', 8, 7, 1),
 (9, 'clean_last_review_c1', 9, 8, 1)]

In [209]:
#dcm1.curr_col = original_pd.columns
#dcm1.curr_row = list(original_pd.index) 
#dcm1.curr_df = original_pd

In [210]:
#for i,c in enumerate(change_index.columns):
#    dcm1.pd_index.loc[change_index.loc[:,c].values,i]

In [211]:
def assign_to_original(original_pd,workset_pd):
    original_pd.loc[original_pd.id.isin(workset_pd.id),:] = workset_pd
    return original_pd

In [212]:
airbnb_pd = pd.read_csv("../../../collaboration_simulation/airbnb_test_case/chicago_vert_2_a.csv",dtype=str)
for x in list(filter(lambda x:x.endswith("flag"),airbnb_pd.columns)):
    airbnb_pd.loc[:,x] = airbnb_pd.loc[:,x].astype(int) 
airbnb_pd.index = group2_index

/tmp/ipykernel_3425/3739463790.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  airbnb_pd.loc[:,x] = airbnb_pd.loc[:,x].astype(int)


In [213]:
airbnb_pd.latitude.sort_values(ascending=False)

3171    42.01833
1767    42.01653
1759    42.01653
1502    42.01653
1766    42.01653
          ...   
2543         NaN
2557         NaN
2657         NaN
3014         NaN
3136         NaN
Name: latitude, Length: 1715, dtype: object

In [214]:
airbnb_pd.shape,original_pd.shape

((1715, 27), (3201, 27))

In [215]:
# take part of airbnb_pd from original_pd
# using data frame imutability, any changes will be 
# directed toward those subset of original_pd
#airbnb_pd = original_pd[original_pd.id.isin(airbnb_pd.id)]

# cleanup duplicate id
The ID column must contain unique values. If there are any duplicate values in this column, you will need to take action to ensure that each ID is unique. You can do this by either fixing the duplicates (if you want to keep them) or by flagging them for removal (1) using the id_flag column.

In [216]:
def clean_duplicate_id(df):
    dup_ids = df[df.id_flag==0]
    dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
    dup_ids = dup_ids[dup_ids.name>1]
    
    for id_ in list(dup_ids["id"]):
        temp_df=df[df["id"]==id_]
        
        try:
            id_=list(temp_df[temp_df.duplicated()].index)[0]  ## two rows are completely the same
            df.loc[id_,"id_flag"]=1
        except:
            index_ls=list(temp_df.index)   ## one row combine with another row equals to one of the original row
            temp_row_1=temp_df.loc[index_ls[0]]
            temp_row_2=temp_df.loc[index_ls[1]]
            row_comb=temp_df.loc[index_ls[0]].combine_first(temp_df.loc[index_ls[1]])
            try:
                comp_result_1=list(set(list(temp_row_1.fillna(0)==row_comb.fillna(0))))[0]
                comp_result_2=list(set(list(temp_row_2.fillna(0)==row_comb.fillna(0))))[0]
                if comp_result_1==True:
                    df.loc[index_ls[1],"id_flag"]=1
                elif comp_result_2==True:
                    df.loc[index_ls[0],"id_flag"]=1
                else:
                    df.loc[index_ls[0],"id_flag"]=2
                    df.loc[index_ls[1],"id_flag"]=2  ##mark types which not belongs to the types mentioned above
            except:
                print(id_)
    return df

In [217]:
dup_ids = airbnb_pd[airbnb_pd.id_flag==0]
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]

problem_n = dup_ids.shape[0]
dq_problem = problem_n/problem_n
problem_n,dq_problem

(69, 1.0)

In [218]:
airbnb_pd = clean_duplicate_id(airbnb_pd)
airbnb_pd

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
1486,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,85,...,R21000075737,0,0,0,0,0,0,0,0,0
1487,84042,The Explorer Room,334241,Michael And Veronica,NaN,West Town,4189615,-87.67934,Private room,75,...,R17000013986,0,0,0,0,0,0,0,0,0
1488,84042,The Explorer Room,334241,Michael And Veronica,NaN,West Town,4189615,-87.67934,Private room,75,...,R17000013986,1,0,0,0,0,0,0,0,0
1489,145659,Trendy Roscoe Village 3BR/2BR walk to shops,683529,Joe,NaN,North Center,41.94342,-87.68121,Entire home/apt,205,...,2209272,0,0,0,0,0,0,0,0,0
1490,264547,Steps From the Best Chicago Offers,1387948,Dominic,NaN,Near West Side,41.88467,-87.64487,Entire home/apt,204,...,City registration pending,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,782628636832878491,"Steps to Shop, Eat, Train | Easy Access | Zencity",47172572,Zencity,NaN,West Town,41.8955,-87.66124,Entire home/apt,50,...,R19000043484,0,0,0,0,0,0,0,0,0
3197,782643895516370805,Old Town Oasis,169297663,William,NaN,Near North Side,41.90105,-87.63716,Apartment,120,...,R22000093645,0,0,0,0,0,0,0,0,0
3198,784994899201350568,Lovely 1 bed Apt in River North,52827024,Yakir,NaN,Near West Side,41.88822006416301,-87.64145321718578,Entire home/apt,84,...,NaN,0,0,0,0,0,0,0,0,0
3199,785423932330914663,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,NaN,Near North Side,41.890516,-87.635955,Apartment,169,...,NaN,0,0,0,0,0,0,0,0,0


# Duplicate IDS checking 
To ensure that all ID values in the dataset are unique, you should check for duplicate IDs. When you run the query to check for duplicates, there should be no rows returned, indicating that there are no duplicate ID values present in the dataset.

In [219]:
dup_ids = airbnb_pd
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]
removal_flag = (airbnb_pd.id_flag!=0).sum()
corrected = dup_ids.shape[0]


In [220]:
removal_flag

69

In [221]:
completeness = (problem_n-corrected) / problem_n
flagged = removal_flag / problem_n
completeness,flagged


(0.0, 1.0)

In [222]:
dup_ids = airbnb_pd[airbnb_pd.id_flag==0]
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]
dup_ids


,id,name


In [223]:
dq_post = dup_ids.shape[0]/problem_n
dq_post

0.0

In [224]:
workset_before = original_pd.copy()
workset_after = original_pd.copy()

In [225]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [226]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                 0
name                               0
host_id                            0
host_name                          0
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
number_of_reviews_ltm              0
license                            0
id_flag                           69
host_id_flag                       0
neighbourhood_flag                 0
latitude_flag                      0
longitude_flag                     0
minimum_nights_flag                0
number_of_reviews_flag             0
last_review_flag                   0
room_type_flag                     0
d

In [227]:
dcm1.change_df(workset_before,workset_after,"clean_duplicate_id_c2","c2",0)

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40194, 40194, 1488, 18), (42003, 42003, 1555, 18), (43542, 43542, 1612, 18), (43893, 43893, 1625, 18), (44055, 44055, 1631, 18), (44595, 44595, 1651, 18), (44838, 44838, 1660, 18), (45270, 45270, 1676, 18), (45324, 45324, 1678, 18), (45405, 45405, 1681, 18), (45918, 45918, 1700, 18), (46242, 46242, 1712, 18), (47025, 47025, 1741, 18), (47295, 47295, 1751, 18), (47997, 47997, 1777, 18), (50535, 50535, 1871, 18), (50778, 50778, 1880, 18), (51048, 51048, 1890, 18), (51399, 51399, 1903, 18), (51588, 51588, 1910, 18), (52344, 52344, 1938, 18), (52398, 52398, 1940, 18), (52560, 52560, 1946, 18), (52965, 52965, 1961, 18), (53100, 53100, 1966, 18), (53856, 53856, 1994, 18), (53964, 53964, 1998, 18), (54693, 54693, 2025, 18), (54801, 54801, 2029, 18), (56313, 56313, 2085, 18), (57069, 57069, 2113, 18), (59337, 59337, 2197, 18), (59472, 59472, 2202, 18), (59580, 59580, 2206, 18), (60903, 60903, 2255, 18), (62739, 62739, 2323, 18), (63279, 63279, 2343, 18),

True

In [228]:
dcm1.state,dcm1.user,dcm1.state_detail

([(-1, -2),
  (0, -1),
  (1, 0),
  (2, 1),
  (3, 2),
  (4, 3),
  (5, 4),
  (6, 5),
  (7, 6),
  (8, 7),
  (9, 8),
  (10, 0)],
 {None: -1, 's1': 0, 'c1': 1, 'c2': 2},
 [(-1, {'op': 'initial'}, 0, -2, None),
  (0, 'perturbed_dataset', 0, -1, 0),
  (1, 'clean_duplicate_id_c1', 1, 0, 1),
  (2, 'clean_inconsistent_host_id_c1', 2, 1, 1),
  (3, 'clean_neighbourhood_c1', 3, 2, 1),
  (4, 'clean_latitude_c1', 4, 3, 1),
  (5, 'clean_longitude_c1', 5, 4, 1),
  (6, 'clean_room_type_c1', 6, 5, 1),
  (7, 'clean_minimum_nights_c1', 7, 6, 1),
  (8, 'clean_number_of_reviews_c1', 8, 7, 1),
  (9, 'clean_last_review_c1', 9, 8, 1),
  (10, 'clean_duplicate_id_c2', 10, 0, 2)])

In [229]:
#dcm1.change_df(workset_before,workset_after,"clean_duplicate_id_c2","c2",1)

In [230]:
workset_before = workset_after.copy()

In [231]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==0]

,0,1,2,3,4
86427,86427,66,0,"November 28, 2022",66
86428,86428,89,0,Entire home,89
86429,86429,167,0,Lhke eieu,167
86430,86430,170,0,Entire home,170
86431,86431,224,0,Entire home,224
...,...,...,...,...,...
88713,88713,86246,0,Apartment,86246
88714,88714,86273,0,Apartment,86273
88715,88715,86327,0,Apartment,86327
88716,88716,86381,0,Apartment,86381


# cleanup inconsistent host id
Each host_id value in the dataset should be associated with only one host_name. However, there may be inconsistencies in the dataset where a host_id is associated with different host_name values.

To clean this up, you can either change the host_name value to a consistent value based on information in the dataset, or flag the host_id_flag column to indicate that the row should be removed from downstream tasks.

For example, if you find that a host_id is associated with multiple host_name values, you may want to investigate further to determine which host_name is correct. If one of the host_name values is clearly incorrect (e.g., a misspelling or a name that does not match the owner of the property), you could update the host_name value to the correct value.

Alternatively, if you cannot determine the correct host_name value, or if you want to exclude the row from downstream tasks for other reasons, you can flag the host_id_flag column with a value of 1 to indicate that the row should be removed.

In [232]:
dup_host_id = airbnb_pd[airbnb_pd.host_id_flag==0]
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()

problem_n = dup_host_id.shape[0]
dq_problem = problem_n/problem_n
problem_n,dq_problem

(931, 1.0)

In [233]:
import re

def clean_host_id(df):
    dup_host_id = df[df.host_id_flag==0]
    dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
    dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
    dup_host_id_df=dup_host_id[dup_host_id["id"]>1]
    dup_host_id_ls=list(dup_host_id_df["host_id"])
    for dup_host_id in dup_host_id_ls:
        temp_df=df[df["host_id"]==dup_host_id]
        temp_id_ls=list(temp_df.index)
        
        temp_match_ls=[]
        for id_ in temp_id_ls:
            string=temp_df.loc[id_,"host_name"]
            pattern = r"^[A-Z][a-z0-9_-]{3,19}$"  # here is only a simple way to clean the host_id
            match=re.match(pattern, string)
            if match:
                temp_match_ls.append(string)
            else:
                df.loc[id_,"host_id_flag"]=1  # while this method is also not perfect, since under same host_id, the host_name could be all wrong
        
        if len(set(temp_match_ls))!=1:
            for id_ in temp_id_ls:
                 df.loc[id_,"host_id_flag"]=2
    return df

In [234]:
airbnb_pd = clean_host_id(airbnb_pd)

# Inconsistent Host ID checking 

This query should return zero rows once you implement the cleaning process

In [235]:
dup_host_id = airbnb_pd
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
dup_host_id[dup_host_id["id"]>1]
removal_flag = (airbnb_pd.host_id_flag!=0).sum()
corrected = dup_host_id.shape[0]


completeness = (problem_n-corrected) / problem_n
flagged = removal_flag / (problem_n/2)
completeness,flagged

(0.0, 0.4597207303974221)

In [236]:
dup_host_id = airbnb_pd[airbnb_pd.host_id_flag==0]
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
dup_host_id[dup_host_id["id"]>1]

,host_id,id


In [237]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [238]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                         0
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
id_flag                             0
host_id_flag                      214
neighbourhood_flag                  0
latitude_flag                       0
longitude_flag                      0
minimum_nights_flag                 0
number_of_reviews_flag              0
last_review_flag                    0
room_type_fl

In [239]:
dcm1.change_df(workset_before,workset_after,"clean_inconsistent_host_id_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40573, 40573, 1502, 19), (41356, 41356, 1531, 19), (41653, 41653, 1542, 19), (41842, 41842, 1549, 19), (41869, 41869, 1550, 19), (41977, 41977, 1554, 19), (42004, 42004, 1555, 19), (42058, 42058, 1557, 19), (42517, 42517, 1574, 19), (42625, 42625, 1578, 19), (42652, 42652, 1579, 19), (42679, 42679, 1580, 19), (42733, 42733, 1582, 19), (43057, 43057, 1594, 19), (43246, 43246, 1601, 19), (43354, 43354, 1605, 19), (43381, 43381, 1606, 19), (43624, 43624, 1615, 19), (43786, 43786, 1621, 19), (43948, 43948, 1627, 19), (43975, 43975, 1628, 19), (44272, 44272, 1639, 19), (44569, 44569, 1650, 19), (44596, 44596, 1651, 19), (45082, 45082, 1669, 19), (45244, 45244, 1675, 19), (45271, 45271, 1676, 19), (45514, 45514, 1685, 19), (45595, 45595, 1688, 19), (45703, 45703, 1692, 19), (45865, 45865, 1698, 19), (46216, 46216, 1711, 19), (46243, 46243, 1712, 19), (46297, 46297, 1714, 19), (46486, 46486, 1721, 19), (46891, 46891, 1736, 19), (46918, 46918, 1737, 19),

DEBUG:root:('change_values idxlist:', ((43948, 43948, 1627, 19), 2.0, (1627, 19)))
DEBUG:root:('change_values idxlist:', ((43975, 43975, 1628, 19), 2.0, (1628, 19)))
DEBUG:root:('change_values idxlist:', ((44272, 44272, 1639, 19), 2.0, (1639, 19)))
DEBUG:root:('change_values idxlist:', ((44569, 44569, 1650, 19), 2.0, (1650, 19)))
DEBUG:root:('change_values idxlist:', ((44596, 44596, 1651, 19), 2.0, (1651, 19)))
DEBUG:root:('change_values idxlist:', ((45082, 45082, 1669, 19), 1.0, (1669, 19)))
DEBUG:root:('change_values idxlist:', ((45244, 45244, 1675, 19), 2.0, (1675, 19)))
DEBUG:root:('change_values idxlist:', ((45271, 45271, 1676, 19), 2.0, (1676, 19)))
DEBUG:root:('change_values idxlist:', ((45514, 45514, 1685, 19), 2.0, (1685, 19)))
DEBUG:root:('change_values idxlist:', ((45595, 45595, 1688, 19), 2.0, (1688, 19)))
DEBUG:root:('change_values idxlist:', ((45703, 45703, 1692, 19), 2.0, (1692, 19)))
DEBUG:root:('change_values idxlist:', ((45865, 45865, 1698, 19), 2.0, (1698, 19)))
DEBU

DEBUG:root:('change_values idxlist:', ((67924, 67924, 2515, 19), 1.0, (2515, 19)))
DEBUG:root:('change_values idxlist:', ((68626, 68626, 2541, 19), 2.0, (2541, 19)))
DEBUG:root:('change_values idxlist:', ((68869, 68869, 2550, 19), 2.0, (2550, 19)))
DEBUG:root:('change_values idxlist:', ((68896, 68896, 2551, 19), 2.0, (2551, 19)))
DEBUG:root:('change_values idxlist:', ((69517, 69517, 2574, 19), 2.0, (2574, 19)))
DEBUG:root:('change_values idxlist:', ((69922, 69922, 2589, 19), 1.0, (2589, 19)))
DEBUG:root:('change_values idxlist:', ((69949, 69949, 2590, 19), 1.0, (2590, 19)))
DEBUG:root:('change_values idxlist:', ((70786, 70786, 2621, 19), 1.0, (2621, 19)))
DEBUG:root:('change_values idxlist:', ((70813, 70813, 2622, 19), 2.0, (2622, 19)))
DEBUG:root:('change_values idxlist:', ((70840, 70840, 2623, 19), 2.0, (2623, 19)))
DEBUG:root:('change_values idxlist:', ((70867, 70867, 2624, 19), 2.0, (2624, 19)))
DEBUG:root:('change_values idxlist:', ((71002, 71002, 2629, 19), 2.0, (2629, 19)))
DEBU

True

In [240]:
dcm1.state_detail

[(-1, {'op': 'initial'}, 0, -2, None),
 (0, 'perturbed_dataset', 0, -1, 0),
 (1, 'clean_duplicate_id_c1', 1, 0, 1),
 (2, 'clean_inconsistent_host_id_c1', 2, 1, 1),
 (3, 'clean_neighbourhood_c1', 3, 2, 1),
 (4, 'clean_latitude_c1', 4, 3, 1),
 (5, 'clean_longitude_c1', 5, 4, 1),
 (6, 'clean_room_type_c1', 6, 5, 1),
 (7, 'clean_minimum_nights_c1', 7, 6, 1),
 (8, 'clean_number_of_reviews_c1', 8, 7, 1),
 (9, 'clean_last_review_c1', 9, 8, 1),
 (10, 'clean_duplicate_id_c2', 10, 0, 2),
 (11, 'clean_inconsistent_host_id_c2', 11, 10, 2)]

In [241]:
workset_before = workset_after.copy()

In [242]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==1]

,0,1,2,3,4
88718,88718,423,1,1,423
88719,88719,2313,1,1,2313
88720,88720,3420,1,1,3420
88721,88721,4122,1,1,4122
88722,88722,4392,1,1,4392
...,...,...,...,...,...
88783,88783,35037,1,1,35037
88784,88784,35280,1,1,35280
88785,88785,36522,1,1,36522
88786,88786,36630,1,1,36630


# cleanup neighbourhood
The neighbourhood column in the dataset should contain values that match the neighbourhoods defined in the official neighbourhood_list. However, there may be some values in the neighbourhood column that are incorrect due to errors or noise in the data.

To clean up the neighbourhood column, you can try to match each value in the column to a valid neighbourhood in the neighbourhood_list using a string distance function such as abydos. If you can successfully match a value in the neighbourhood column to a neighbourhood in the neighbourhood_list, you can replace the value in the dataset with the correct neighbourhood name.

However, if you are unsure about how to clean up a particular value in the neighbourhood column, or if you cannot match the value to a valid neighbourhood in the neighbourhood_list, you can flag the row for deletion by setting the neighbourhood_flag column to a value of 1. If the value in the neighbourhood column is null and you cannot make a determination based on other information in the dataset, you can set the neighbourhood_flag column to a value of 2 to indicate that the row should be included but the neighbourhood value is null.

You can also use the latitude and longitude columns in the dataset to help match values in the neighbourhood column to valid neighbourhoods in the neighbourhood_list. However, you should be aware that the latitude and longitude values may also contain errors or noise, so you should exercise caution when using these columns to clean up the neighbourhood column.

In [243]:
neighbourhood_list = [ 'Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View',    'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater',    'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park',    'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side',    'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood',    'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale',    'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park',    'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore',    'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa',    'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park',    'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn',    'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights',    'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn',    'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland',    'West Englewood', 'Calumet Heights', 'Auburn Gresham', 'Fuller Park',    'Avalon Park', 'Burnside', 'Clearing', 'Gage Park', 'West Pullman',    'Washington Heights', 'East Side']
print(neighbourhood_list)

['Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View', 'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater', 'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park', 'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side', 'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood', 'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale', 'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park', 'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore', 'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa', 'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park', 'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn', 'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights', 'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn', 'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland', 'West Englewood', 'Calu

In [244]:
from abydos import distance

In [245]:
#distance

In [246]:
airbnb_pd

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
1486,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,85,...,R21000075737,0,0,0,0,0,0,0,0,0
1487,84042,The Explorer Room,334241,Michael And Veronica,NaN,West Town,4189615,-87.67934,Private room,75,...,R17000013986,0,0,0,0,0,0,0,0,0
1488,84042,The Explorer Room,334241,Michael And Veronica,NaN,West Town,4189615,-87.67934,Private room,75,...,R17000013986,1,0,0,0,0,0,0,0,0
1489,145659,Trendy Roscoe Village 3BR/2BR walk to shops,683529,Joe,NaN,North Center,41.94342,-87.68121,Entire home/apt,205,...,2209272,0,0,0,0,0,0,0,0,0
1490,264547,Steps From the Best Chicago Offers,1387948,Dominic,NaN,Near West Side,41.88467,-87.64487,Entire home/apt,204,...,City registration pending,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,782628636832878491,"Steps to Shop, Eat, Train | Easy Access | Zencity",47172572,Zencity,NaN,West Town,41.8955,-87.66124,Entire home/apt,50,...,R19000043484,0,0,0,0,0,0,0,0,0
3197,782643895516370805,Old Town Oasis,169297663,William,NaN,Near North Side,41.90105,-87.63716,Apartment,120,...,R22000093645,0,0,0,0,0,0,0,0,0
3198,784994899201350568,Lovely 1 bed Apt in River North,52827024,Yakir,NaN,Near West Side,41.88822006416301,-87.64145321718578,Entire home/apt,84,...,NaN,0,0,0,0,0,0,0,0,0
3199,785423932330914663,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,NaN,Near North Side,41.890516,-87.635955,Apartment,169,...,NaN,0,0,0,0,0,0,0,0,0


In [247]:
def clean_neighbourhood(df):
    neighbourhood_check = df[df.neighbourhood_flag==0]
    neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
    neigh_id_df=neighbourhood_check[["id","neighbourhood"]]
    neigh_ind_ls=list(neigh_id_df.index)
    
    for neigh_ind in neigh_ind_ls:
        sus_neigh_ipt=df.loc[neigh_ind,"neighbourhood"]

        try:
            
            temp_ls=[]
            for neighbour in neighbourhood_list:
                temp_ls.append(distance.sim(sus_neigh_ipt, neighbour))
            if max(temp_ls)>0.5:
                df.loc[neigh_ind,"neighbourhood"]=neighbourhood_list[temp_ls.index(max(temp_ls))]
            else:
                df.loc[neigh_ind,"neighbourhood_flag"]=1
        except BaseException as ex:
            print(ex)
            df.loc[neigh_ind,"neighbourhood_flag"]=2
        
    return df

In [248]:
neighbourhood_check = airbnb_pd[airbnb_pd.neighbourhood_flag==0]
neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

problem_n = neighbourhood_check.shape[0]
dq_problem = problem_n/problem_n
problem_n,dq_problem

(373, 1.0)

In [249]:
airbnb_pd = clean_neighbourhood(airbnb_pd)

object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()


In [250]:
airbnb_pd[airbnb_pd['neighbourhood_flag']==2]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,license,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
1712,20053035,Modern Chicago Loft with a Complimentary Garag...,117498144,Kashao& Owen,NaN,NaN,41.94139,-87.73096,Apartment,45,...,“City registration pending”,1,2,2,0,0,0,0,0,0
2054,37198925,NaN,22580828,Derek,NaN,NaN,41.88663,-87.64395,Apartment,64,...,City registration pending,0,0,2,0,0,0,0,0,0
2747,604136671933317555,NaN,47172572,Zencity,NaN,NaN,41.90278,-87.69213,Entire home/apt,NaN,...,R18000031772,0,0,2,0,0,0,0,0,0
2994,711841142491970819,NaN,395498979,Milan,NaN,NaN,41.85525,-87.62515,Entire home/apt,136,...,2792126,0,2,2,0,0,0,0,0,0
3014,717673204137659686,Pilsen Paradise| 3 miles to DT | Sleeps 8,431261724,AW Luxe,NaN,NaN,NaN,-87.66591,Entire home/apt,114,...,R21000071936,1,2,2,0,0,0,0,0,0
3136,760221018437590687,NaN,427604956,Abdullah,NaN,NaN,NaN,-87.62651,Entire home/apt,140,...,2882270,0,0,2,0,0,0,0,0,0


# Neighbourhood checking

This query should return zero rows once you implement the cleaning process

In [251]:
neighbourhood_check = airbnb_pd
neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

removal_flag = (airbnb_pd.neighbourhood_flag!=0).sum()
corrected = neighbourhood_check.shape[0]

completeness = (problem_n-corrected) / problem_n
flagged = removal_flag / (problem_n/2)
completeness,flagged

(0.9839142091152815, 0.032171581769437)

In [252]:
neighbourhood_check = airbnb_pd[airbnb_pd.neighbourhood_flag==0]
neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

,id,neighbourhood


In [253]:
workset_before = workset_after.copy()

In [254]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [255]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                     367
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                         0
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
id_flag                             0
host_id_flag                        0
neighbourhood_flag                  6
latitude_flag                       0
longitude_flag                      0
minimum_nights_flag                 0
number_of_reviews_flag              0
last_review_flag                    0
room_type_fl

In [256]:
dcm1.change_df(workset_before,workset_after,"clean_neighbourhood_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40343, 87489, 1494, 5), (40397, 87491, 1496, 5), (40505, 87494, 1500, 5), (40613, 87495, 1504, 5), (40775, 87499, 1510, 5), (40802, 87500, 1511, 5), (40910, 87503, 1515, 5), (40937, 87504, 1516, 5), (41153, 87509, 1524, 5), (41342, 87513, 1531, 5), (41396, 87514, 1533, 5), (41423, 87515, 1534, 5), (41477, 87516, 1536, 5), (41666, 87524, 1543, 5), (41693, 87525, 1544, 5), (41747, 87526, 1546, 5), (41774, 87527, 1547, 5), (41963, 87533, 1554, 5), (41990, 87535, 1555, 5), (42125, 87539, 1560, 5), (42233, 87542, 1564, 5), (42449, 87547, 1572, 5), (42557, 87548, 1576, 5), (42584, 87550, 1577, 5), (42908, 87553, 1589, 5), (42962, 87554, 1591, 5), (42989, 87556, 1592, 5), (43043, 87559, 1594, 5), (43097, 87560, 1596, 5), (43205, 87561, 1600, 5), (43367, 87567, 1606, 5), (43610, 87572, 1615, 5), (43745, 87574, 1620, 5), (43961, 87582, 1628, 5), (44177, 87587, 1636, 5), (44312, 87590, 1641, 5), (44663, 87596, 1654, 5), (44798, 87597, 1659, 5), (44825, 876

DEBUG:root:('change_values idxlist:', ((40343, 87489, 1494, 5), 'West Town', (1494, 5)))
DEBUG:root:('change_values idxlist:', ((40397, 87491, 1496, 5), 'Edgewater', (1496, 5)))
DEBUG:root:('change_values idxlist:', ((40505, 87494, 1500, 5), 'West Town', (1500, 5)))
DEBUG:root:('change_values idxlist:', ((40613, 87495, 1504, 5), 'West Town', (1504, 5)))
DEBUG:root:('change_values idxlist:', ((40775, 87499, 1510, 5), 'Near South Side', (1510, 5)))
DEBUG:root:('change_values idxlist:', ((40802, 87500, 1511, 5), 'Morgan Park', (1511, 5)))
DEBUG:root:('change_values idxlist:', ((40910, 87503, 1515, 5), 'Near South Side', (1515, 5)))
DEBUG:root:('change_values idxlist:', ((40937, 87504, 1516, 5), 'West Town', (1516, 5)))
DEBUG:root:('change_values idxlist:', ((41153, 87509, 1524, 5), 'Armour Square', (1524, 5)))
DEBUG:root:('change_values idxlist:', ((41342, 87513, 1531, 5), 'West Town', (1531, 5)))
DEBUG:root:('change_values idxlist:', ((41396, 87514, 1533, 5), 'West Town', (1533, 5)))
DEB

DEBUG:root:('change_values idxlist:', ((50279, 87742, 1862, 5), 'Near West Side', (1862, 5)))
DEBUG:root:('change_values idxlist:', ((50576, 87748, 1873, 5), 'Near North Side', (1873, 5)))
DEBUG:root:('change_values idxlist:', ((50603, 87750, 1874, 5), 'Humboldt Park', (1874, 5)))
DEBUG:root:('change_values idxlist:', ((50738, 87754, 1879, 5), 'Near North Side', (1879, 5)))
DEBUG:root:('change_values idxlist:', ((50765, 87755, 1880, 5), 'Near North Side', (1880, 5)))
DEBUG:root:('change_values idxlist:', ((50954, 87757, 1887, 5), 'Near North Side', (1887, 5)))
DEBUG:root:('change_values idxlist:', ((51089, 87758, 1892, 5), 'North Lawndale', (1892, 5)))
DEBUG:root:('change_values idxlist:', ((51224, 87761, 1897, 5), 'Near North Side', (1897, 5)))
DEBUG:root:('change_values idxlist:', ((51386, 87767, 1903, 5), 'Near South Side', (1903, 5)))
DEBUG:root:('change_values idxlist:', ((51413, 87770, 1904, 5), 'Near South Side', (1904, 5)))
DEBUG:root:('change_values idxlist:', ((51548, 87771, 

DEBUG:root:('change_values idxlist:', ((62429, 88060, 2312, 5), 'Near South Side', (2312, 5)))
DEBUG:root:('change_values idxlist:', ((62510, 88061, 2315, 5), 'Hermosa', (2315, 5)))
DEBUG:root:('change_values idxlist:', ((62672, 88065, 2321, 5), 'Edgewater', (2321, 5)))
DEBUG:root:('change_values idxlist:', ((62753, 88070, 2324, 5), 'Near North Side', (2324, 5)))
DEBUG:root:('change_values idxlist:', ((62888, 88073, 2329, 5), 'Near North Side', (2329, 5)))
DEBUG:root:('change_values idxlist:', ((62969, 88077, 2332, 5), 'Near North Side', (2332, 5)))
DEBUG:root:('change_values idxlist:', ((63212, 88084, 2341, 5), 'Near North Side', (2341, 5)))
DEBUG:root:('change_values idxlist:', ((63590, 88087, 2355, 5), 'Near West Side', (2355, 5)))
DEBUG:root:('change_values idxlist:', ((63671, 88089, 2358, 5), 'West Ridge', (2358, 5)))
DEBUG:root:('change_values idxlist:', ((63887, 88092, 2366, 5), 'Near North Side', (2366, 5)))
DEBUG:root:('change_values idxlist:', ((63941, 88095, 2368, 5), 'Humbo

DEBUG:root:('change_values idxlist:', ((73121, 88348, 2708, 5), 'Irving Park', (2708, 5)))
DEBUG:root:('change_values idxlist:', ((73310, 88353, 2715, 5), 'Hermosa', (2715, 5)))
DEBUG:root:('change_values idxlist:', ((73364, 88355, 2717, 5), 'Mckinley Park', (2717, 5)))
DEBUG:root:('change_values idxlist:', ((73499, 88359, 2722, 5), 'Near West Side', (2722, 5)))
DEBUG:root:('change_values idxlist:', ((73580, 88360, 2725, 5), 'Near North Side', (2725, 5)))
DEBUG:root:('change_values idxlist:', ((73715, 88364, 2730, 5), 'Oakland', (2730, 5)))
DEBUG:root:('change_values idxlist:', ((73742, 88365, 2731, 5), 'Near West Side', (2731, 5)))
DEBUG:root:('change_values idxlist:', ((73796, 88368, 2733, 5), 'Bridgeport', (2733, 5)))
DEBUG:root:('change_values idxlist:', ((73823, 88369, 2734, 5), 'Bridgeport', (2734, 5)))
DEBUG:root:('change_values idxlist:', ((74066, 88375, 2743, 5), 'Irving Park', (2743, 5)))
DEBUG:root:('change_values idxlist:', ((74120, 88377, 2745, 5), 'North Center', (2745, 5

DEBUG:root:('change_values idxlist:', ((84623, 88658, 3134, 5), 'Near West Side', (3134, 5)))
DEBUG:root:('change_values idxlist:', ((84692, 84692, 3136, 20), 2.0, (3136, 20)))
DEBUG:root:('change_values idxlist:', ((84920, 88669, 3145, 5), 'Near West Side', (3145, 5)))
DEBUG:root:('change_values idxlist:', ((85055, 88673, 3150, 5), 'West Town', (3150, 5)))
DEBUG:root:('change_values idxlist:', ((85136, 88675, 3153, 5), 'West Town', (3153, 5)))
DEBUG:root:('change_values idxlist:', ((85163, 88676, 3154, 5), 'Kenwood', (3154, 5)))
DEBUG:root:('change_values idxlist:', ((85190, 88679, 3155, 5), 'Kenwood', (3155, 5)))
DEBUG:root:('change_values idxlist:', ((85271, 88683, 3158, 5), 'South Chicago', (3158, 5)))
DEBUG:root:('change_values idxlist:', ((85460, 88689, 3165, 5), 'Near North Side', (3165, 5)))
DEBUG:root:('change_values idxlist:', ((85730, 88693, 3175, 5), 'Near West Side', (3175, 5)))
DEBUG:root:('change_values idxlist:', ((85757, 88694, 3176, 5), 'Near West Side', (3176, 5)))
D

True

In [257]:
workset_before = workset_after.copy()

In [258]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==2]

,0,1,2,3,4
88788,88788,721,2,1,721
88789,88789,748,2,1,748
88790,88790,1423,2,1,1423
88791,88791,1450,2,1,1450
88792,88792,1477,2,1,1477
...,...,...,...,...,...
89137,89137,39871,2,1,39871
89138,89138,39979,2,1,39979
89139,89139,40006,2,1,40006
89140,89140,40033,2,1,40033


# cleanup latitude and longitude
The latitude and longitude values in the dataset must fall within the range of -90 to +90 for latitude and -180 to +180 for longitude to ensure that they meet the criteria for analysis. We have provided a check number function to validate the latitude and longitude columns. Any values outside of these ranges should be cleaned to meet the criteria.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting latitude_flag or longitude_flag to 1 or 2, respectively.

In [259]:
def check_number(x,start=-90,end=90):
    try:
        temp_x = float(x)
        return (start <= temp_x <= end)
    except:
        return False

In [260]:
def clean_latitude(df):
    
    lat_check_pd = df[df.latitude_flag==0]
    lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
    wrong_lat_df=lat_check_pd[["id","latitude"]]
    lat_ind_ls=list(wrong_lat_df.index)
    
    lat_true = df[df.latitude_flag==0]
    lat_true = lat_true[lat_true.latitude.apply(lambda x:check_number(x,-90,90))==True]
    right_lati_ls=list(lat_true["latitude"])
    #print(right_lati_ls)
    avg_lati=sum([float(x) for x in right_lati_ls])/len(right_lati_ls)
    uni_period_loc=str(avg_lati).index(".")
    print(uni_period_loc)
    
    
    for lat_id in lat_ind_ls:
        pre_lati=df.loc[lat_id,"latitude"]
        if pd.isnull(pre_lati):
            df.loc[lat_id,"latitude_flag"]=2
        else:
            try:
                temp=[]
                period_loc=0
                for c in str(pre_lati):
                    # nikolaus fixed the isdigit calling function and added checking for "-" sign 
                    if c.isdigit:                    
                    #if c.isdigit() or c=="-":
                        temp.append(c)
                        # nikolaus fixed missplaced =+1 shoud be +=1 (for iteration)
                        period_loc=+1                        
                        #period_loc+=1
                    #print(uni_period_loc,period_loc)
                    if period_loc==uni_period_loc:
                        # nikolaus added implementation for "." (based on the intention)
                        temp.append(c)
                        #temp.append(".")
                #print(temp)
                revised=float("".join(temp))
                if check_number(revised)==True:
                    df.loc[lat_id,"latitude"]=revised
                else:
                    df.loc[lat_id,"latitude_flag"]=1
            except BaseException as ex:
                print(lat_id)
                #df.loc[lat_id,"latitude_flag"]=1
    return df

In [261]:
airbnb_pd.latitude.sort_values(ascending=False)

3171    42.01833
1767    42.01653
1759    42.01653
1502    42.01653
1766    42.01653
          ...   
2543         NaN
2557         NaN
2657         NaN
3014         NaN
3136         NaN
Name: latitude, Length: 1715, dtype: object

In [262]:
# apply the clean_latitude function to the 'latitude' column
airbnb_pd = clean_latitude(airbnb_pd)
# airbnb_pd.loc[airbnb_pd['latitude_flag']==1, 'latitude']
#airbnb_pd.loc[airbnb_pd['latitude_flag']==1, 'latitude'] = airbnb_pd.loc[airbnb_pd['latitude_flag']==1, 'latitude'].apply(lambda x: clean_latitude_pls(x))

2


# Latitude checking

This query should return zero rows once you implement the cleaning process

In [263]:
lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
lat_check_pd[["id","latitude"]]

,id,latitude


In [264]:
workset_before = workset_after.copy()

In [265]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [266]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                         0
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
id_flag                             0
host_id_flag                        0
neighbourhood_flag                  0
latitude_flag                     106
longitude_flag                      0
minimum_nights_flag                 0
number_of_reviews_flag              0
last_review_flag                    0
room_type_fl

In [267]:
dcm1.change_df(workset_before,workset_after,"clean_latitude_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40170, 40170, 1487, 21), (40197, 40197, 1488, 21), (40440, 40440, 1497, 21), (40845, 40845, 1512, 21), (41655, 41655, 1542, 21), (41871, 41871, 1550, 21), (41979, 41979, 1554, 21), (42006, 42006, 1555, 21), (42573, 42573, 1576, 21), (43302, 43302, 1603, 21), (43869, 43869, 1624, 21), (43896, 43896, 1625, 21), (43977, 43977, 1628, 21), (44571, 44571, 1650, 21), (44598, 44598, 1651, 21), (44814, 44814, 1659, 21), (44841, 44841, 1660, 21), (44922, 44922, 1663, 21), (45246, 45246, 1675, 21), (45273, 45273, 1676, 21), (45408, 45408, 1681, 21), (45435, 45435, 1682, 21), (46866, 46866, 1735, 21), (47136, 47136, 1745, 21), (47946, 47946, 1775, 21), (48675, 48675, 1802, 21), (49134, 49134, 1819, 21), (50403, 50403, 1866, 21), (50511, 50511, 1870, 21), (50538, 50538, 1871, 21), (50619, 50619, 1874, 21), (50727, 50727, 1878, 21), (51321, 51321, 1900, 21), (51375, 51375, 1902, 21), (51402, 51402, 1903, 21), (54372, 54372, 2013, 21), (54588, 54588, 2021, 21),

DEBUG:root:('change_values idxlist:', ((66090, 66090, 2447, 21), 1.0, (2447, 21)))
DEBUG:root:('change_values idxlist:', ((66225, 66225, 2452, 21), 1.0, (2452, 21)))
DEBUG:root:('change_values idxlist:', ((67008, 67008, 2481, 21), 1.0, (2481, 21)))
DEBUG:root:('change_values idxlist:', ((67656, 67656, 2505, 21), 1.0, (2505, 21)))
DEBUG:root:('change_values idxlist:', ((68682, 68682, 2543, 21), 2.0, (2543, 21)))
DEBUG:root:('change_values idxlist:', ((68898, 68898, 2551, 21), 1.0, (2551, 21)))
DEBUG:root:('change_values idxlist:', ((69060, 69060, 2557, 21), 2.0, (2557, 21)))
DEBUG:root:('change_values idxlist:', ((69357, 69357, 2568, 21), 1.0, (2568, 21)))
DEBUG:root:('change_values idxlist:', ((71760, 71760, 2657, 21), 2.0, (2657, 21)))
DEBUG:root:('change_values idxlist:', ((72813, 72813, 2696, 21), 1.0, (2696, 21)))
DEBUG:root:('change_values idxlist:', ((72840, 72840, 2697, 21), 1.0, (2697, 21)))
DEBUG:root:('change_values idxlist:', ((72867, 72867, 2698, 21), 1.0, (2698, 21)))
DEBU

True

In [268]:
dcm1.state_detail

[(-1, {'op': 'initial'}, 0, -2, None),
 (0, 'perturbed_dataset', 0, -1, 0),
 (1, 'clean_duplicate_id_c1', 1, 0, 1),
 (2, 'clean_inconsistent_host_id_c1', 2, 1, 1),
 (3, 'clean_neighbourhood_c1', 3, 2, 1),
 (4, 'clean_latitude_c1', 4, 3, 1),
 (5, 'clean_longitude_c1', 5, 4, 1),
 (6, 'clean_room_type_c1', 6, 5, 1),
 (7, 'clean_minimum_nights_c1', 7, 6, 1),
 (8, 'clean_number_of_reviews_c1', 8, 7, 1),
 (9, 'clean_last_review_c1', 9, 8, 1),
 (10, 'clean_duplicate_id_c2', 10, 0, 2),
 (11, 'clean_inconsistent_host_id_c2', 11, 10, 2),
 (12, 'clean_neighbourhood_c2', 12, 11, 2),
 (13, 'clean_latitude_c2', 13, 12, 2)]

In [269]:
workset_before = workset_after.copy()

In [270]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==3]

,0,1,2,3,4
89142,89142,167,3,Lake View,86429
89143,89143,248,3,Lincoln Park,86433
89144,89144,329,3,Lincoln Park,86435
89145,89145,356,3,Lincoln Park,86436
89146,89146,383,3,Logan Square,86438
...,...,...,...,...,...
89459,89459,39614,3,Loop,87472
89460,89460,39749,3,Lake View,87476
89461,89461,39884,3,South Shore,87479
89462,89462,39992,3,Loop,87481


In [271]:
def clean_longitude(df):
    lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
    lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
    wrong_long_df=lon_check_pd[["id","longitude"]]
    long_ind_ls=list(wrong_long_df.index)
    
    
    long_true = df[df.longitude_flag==0]
    long_true = long_true[long_true.longitude.apply(lambda x:check_number(x,-180,180))==True]
    right_long_ls=list(long_true["longitude"])
        
    avg_long=sum([float(x) for x in right_long_ls])/len(right_long_ls)
    uni_period_loc=str(avg_long).index(".")

    for long_id in long_ind_ls:
        pre_long=df.loc[long_id,"longitude"]
        if pre_long==np.nan:
            df.loc[long_id,"longitude_flag"]=2
        else:
            try:
                temp=[]
                period_loc=0
                for c in str(pre_long):
                    # nikolaus fixed the isdigit calling function and added checking for "-" sign 
                    if c.isdigit:
                    #if c.isdigit() or c=="-":
                        temp.append(c)
                        # nikolaus fixed missplaced =+1 shoud be +=1 (for iteration)
                        period_loc=+1
                        #period_loc+=1
                    if period_loc==uni_period_loc:
                        # nikolaus added implementation for "." (based on the intention)
                        temp.append(c)
                        #temp.append(".")
                #print(temp)
                revised=float("".join(temp))
                if check_number(revised,-180,180)==True:
                    df.loc[long_id,"longitude"]=revised
                else:
                    df.loc[long_id,"longitude_flag"]=1
        
            except:
                print(long_id)
                df.loc[long_id,"longitude_flag"]=1
        
    return df

In [272]:
airbnb_pd = clean_longitude(airbnb_pd)
airbnb_pd[airbnb_pd['longitude_flag']==1]['longitude']

1540      -8772881
1563      -8763383
1580      -8768732
1607      -8766189
1623      -8769208
           ...    
3083    -876526915
3098      -8766632
3113      -8762559
3127      -8772144
3177      -8763165
Name: longitude, Length: 95, dtype: object

# Longitude checking

This query should return zero rows once you implement the cleaning process

In [273]:
lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
lon_check_pd[["id","longitude"]]

,id,longitude


In [274]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [275]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                 0
name                               0
host_id                            0
host_name                          0
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
number_of_reviews_ltm              0
license                            0
id_flag                            0
host_id_flag                       0
neighbourhood_flag                 0
latitude_flag                      0
longitude_flag                    95
minimum_nights_flag                0
number_of_reviews_flag             0
last_review_flag                   0
room_type_flag                     0
d

In [276]:
dcm1.change_df(workset_before,workset_after,"clean_longitude_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(41602, 41602, 1540, 22), (42223, 42223, 1563, 22), (42682, 42682, 1580, 22), (43411, 43411, 1607, 22), (43843, 43843, 1623, 22), (45652, 45652, 1690, 22), (45679, 45679, 1691, 22), (48028, 48028, 1778, 22), (49108, 49108, 1818, 22), (49189, 49189, 1821, 22), (50296, 50296, 1862, 22), (50593, 50593, 1873, 22), (51187, 51187, 1895, 22), (51322, 51322, 1900, 22), (51646, 51646, 1912, 22), (51673, 51673, 1913, 22), (52321, 52321, 1937, 22), (52348, 52348, 1938, 22), (53374, 53374, 1976, 22), (53752, 53752, 1990, 22), (53833, 53833, 1993, 22), (53860, 53860, 1994, 22), (54670, 54670, 2024, 22), (54697, 54697, 2025, 22), (54778, 54778, 2028, 22), (54805, 54805, 2029, 22), (55777, 55777, 2065, 22), (55939, 55939, 2071, 22), (56128, 56128, 2078, 22), (56398, 56398, 2088, 22), (57451, 57451, 2127, 22), (57505, 57505, 2129, 22), (57856, 57856, 2142, 22), (58666, 58666, 2172, 22), (58693, 58693, 2173, 22), (59557, 59557, 2205, 22), (59584, 59584, 2206, 22),

DEBUG:root:('change_values idxlist:', ((70330, 70330, 2604, 22), 1.0, (2604, 22)))
DEBUG:root:('change_values idxlist:', ((71572, 71572, 2650, 22), 1.0, (2650, 22)))
DEBUG:root:('change_values idxlist:', ((72220, 72220, 2674, 22), 1.0, (2674, 22)))
DEBUG:root:('change_values idxlist:', ((72409, 72409, 2681, 22), 1.0, (2681, 22)))
DEBUG:root:('change_values idxlist:', ((72490, 72490, 2684, 22), 1.0, (2684, 22)))
DEBUG:root:('change_values idxlist:', ((72571, 72571, 2687, 22), 1.0, (2687, 22)))
DEBUG:root:('change_values idxlist:', ((72679, 72679, 2691, 22), 1.0, (2691, 22)))
DEBUG:root:('change_values idxlist:', ((73057, 73057, 2705, 22), 1.0, (2705, 22)))
DEBUG:root:('change_values idxlist:', ((73246, 73246, 2712, 22), 1.0, (2712, 22)))
DEBUG:root:('change_values idxlist:', ((73624, 73624, 2726, 22), 1.0, (2726, 22)))
DEBUG:root:('change_values idxlist:', ((75136, 75136, 2782, 22), 1.0, (2782, 22)))
DEBUG:root:('change_values idxlist:', ((75190, 75190, 2784, 22), 1.0, (2784, 22)))
DEBU

True

In [277]:
dcm1.state_detail

[(-1, {'op': 'initial'}, 0, -2, None),
 (0, 'perturbed_dataset', 0, -1, 0),
 (1, 'clean_duplicate_id_c1', 1, 0, 1),
 (2, 'clean_inconsistent_host_id_c1', 2, 1, 1),
 (3, 'clean_neighbourhood_c1', 3, 2, 1),
 (4, 'clean_latitude_c1', 4, 3, 1),
 (5, 'clean_longitude_c1', 5, 4, 1),
 (6, 'clean_room_type_c1', 6, 5, 1),
 (7, 'clean_minimum_nights_c1', 7, 6, 1),
 (8, 'clean_number_of_reviews_c1', 8, 7, 1),
 (9, 'clean_last_review_c1', 9, 8, 1),
 (10, 'clean_duplicate_id_c2', 10, 0, 2),
 (11, 'clean_inconsistent_host_id_c2', 11, 10, 2),
 (12, 'clean_neighbourhood_c2', 12, 11, 2),
 (13, 'clean_latitude_c2', 13, 12, 2),
 (14, 'clean_longitude_c2', 14, 13, 2)]

In [278]:
workset_before = workset_after.copy()

In [279]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==4]

,0,1,2,3,4
89464,89464,3624,4,41.92748,86509
89465,89465,5082,4,41.93203,86548
89466,89466,5163,4,41.93123,86551
89467,89467,6216,4,41.91334,86580
89468,89468,6243,4,41.69097,86582
...,...,...,...,...,...
89543,89543,37428,4,41.955908,87410
89544,89544,37455,4,42.00246,87411
89545,89545,37779,4,41.881121,87418
89546,89546,38049,4,41.84067,87430


# cleanup room type
The "room_type" column in the dataset should contain one of the values defined in the list of allowed_room_type provided by the authority: ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']. Any value outside of this list needs to be adjusted to one of the allowed values.

If you are unsure about how to adjust the value or cannot find a suitable value, you can flag the row for deletion by setting the value of room_type_flag to 1. If the "room_type" column has a null value and you cannot decide on an appropriate value, you can set the value of room_type_flag to 2.

In [280]:
allowed_room_type = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']

In [281]:
def clean_room_type(df):
    room_type_pd = df[df.room_type_flag==0]
    room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
    room_type_pd[["id","room_type"]]
    room_id_ls=list(room_type_pd[["id","room_type"]].index)
    
    for room_ind in room_id_ls:
        sus_room_ipt=df.loc[room_ind,"room_type"]

        try:
            temp_ls=[]
            for room in allowed_room_type:
                temp_ls.append(distance.sim(sus_room_ipt, room))
            if max(temp_ls)>0.5:
                df.loc[room_ind,"room_type"]=allowed_room_type[temp_ls.index(max(temp_ls))]
            else:
                df.loc[room_ind,"room_type_flag"]=1
        except:
            df.loc[room_ind,"room_type_flag"]=2
        
    return df

In [282]:
airbnb_pd = clean_room_type(airbnb_pd)

# room_type checking

This query should return zero rows once you implement the cleaning process

In [283]:
room_type_pd = airbnb_pd[airbnb_pd.room_type_flag==0]
room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
room_type_pd[["id","room_type"]]

,id,room_type


In [284]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [285]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                         0
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
id_flag                             0
host_id_flag                        0
neighbourhood_flag                  0
latitude_flag                       0
longitude_flag                      0
minimum_nights_flag                 0
number_of_reviews_flag              0
last_review_flag                    0
room_type_fl

In [286]:
dcm1.change_df(workset_before,workset_after,"clean_room_type_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40769, 40769, 1509, 26), (40877, 40877, 1513, 26), (41093, 41093, 1521, 26), (41228, 41228, 1526, 26), (41633, 41633, 1541, 26), (41660, 41660, 1542, 26), (41822, 41822, 1548, 26), (42065, 42065, 1557, 26), (42983, 42983, 1591, 26), (43307, 43307, 1603, 26), (43712, 43712, 1618, 26), (43820, 43820, 1622, 26), (44036, 44036, 1630, 26), (44063, 44063, 1631, 26), (44522, 44522, 1648, 26), (44819, 44819, 1659, 26), (44846, 44846, 1660, 26), (45413, 45413, 1681, 26), (45521, 45521, 1685, 26), (46034, 46034, 1704, 26), (46142, 46142, 1708, 26), (46169, 46169, 1709, 26), (46196, 46196, 1710, 26), (46223, 46223, 1711, 26), (46250, 46250, 1712, 26), (46682, 46682, 1728, 26), (46709, 46709, 1729, 26), (46817, 46817, 1733, 26), (47006, 47006, 1740, 26), (47033, 47033, 1741, 26), (47546, 47546, 1760, 26), (47816, 47816, 1770, 26), (48734, 48734, 1804, 26), (49328, 49328, 1826, 26), (49409, 49409, 1829, 26), (49571, 49571, 1835, 26), (50084, 50084, 1854, 26),

DEBUG:root:('change_values idxlist:', ((45413, 45413, 1681, 26), 2.0, (1681, 26)))
DEBUG:root:('change_values idxlist:', ((45521, 45521, 1685, 26), 1.0, (1685, 26)))
DEBUG:root:('change_values idxlist:', ((46034, 46034, 1704, 26), 1.0, (1704, 26)))
DEBUG:root:('change_values idxlist:', ((46142, 46142, 1708, 26), 1.0, (1708, 26)))
DEBUG:root:('change_values idxlist:', ((46169, 46169, 1709, 26), 1.0, (1709, 26)))
DEBUG:root:('change_values idxlist:', ((46196, 46196, 1710, 26), 1.0, (1710, 26)))
DEBUG:root:('change_values idxlist:', ((46223, 46223, 1711, 26), 1.0, (1711, 26)))
DEBUG:root:('change_values idxlist:', ((46250, 46250, 1712, 26), 1.0, (1712, 26)))
DEBUG:root:('change_values idxlist:', ((46682, 46682, 1728, 26), 1.0, (1728, 26)))
DEBUG:root:('change_values idxlist:', ((46709, 46709, 1729, 26), 1.0, (1729, 26)))
DEBUG:root:('change_values idxlist:', ((46817, 46817, 1733, 26), 1.0, (1733, 26)))
DEBUG:root:('change_values idxlist:', ((47006, 47006, 1740, 26), 1.0, (1740, 26)))
DEBU

DEBUG:root:('change_values idxlist:', ((66284, 66284, 2454, 26), 1.0, (2454, 26)))
DEBUG:root:('change_values idxlist:', ((66986, 66986, 2480, 26), 1.0, (2480, 26)))
DEBUG:root:('change_values idxlist:', ((67310, 67310, 2492, 26), 1.0, (2492, 26)))
DEBUG:root:('change_values idxlist:', ((67391, 67391, 2495, 26), 1.0, (2495, 26)))
DEBUG:root:('change_values idxlist:', ((67499, 67499, 2499, 26), 2.0, (2499, 26)))
DEBUG:root:('change_values idxlist:', ((68093, 68093, 2521, 26), 1.0, (2521, 26)))
DEBUG:root:('change_values idxlist:', ((68282, 68282, 2528, 26), 1.0, (2528, 26)))
DEBUG:root:('change_values idxlist:', ((68363, 68363, 2531, 26), 1.0, (2531, 26)))
DEBUG:root:('change_values idxlist:', ((68498, 68498, 2536, 26), 1.0, (2536, 26)))
DEBUG:root:('change_values idxlist:', ((68606, 68606, 2540, 26), 1.0, (2540, 26)))
DEBUG:root:('change_values idxlist:', ((68795, 68795, 2547, 26), 1.0, (2547, 26)))
DEBUG:root:('change_values idxlist:', ((69065, 69065, 2557, 26), 2.0, (2557, 26)))
DEBU

DEBUG:root:('change_values idxlist:', ((85994, 85994, 3184, 26), 1.0, (3184, 26)))
DEBUG:root:('change_values idxlist:', ((86264, 86264, 3194, 26), 1.0, (3194, 26)))
DEBUG:root:('change_values idxlist:', ((86291, 86291, 3195, 26), 1.0, (3195, 26)))
DEBUG:root:('change_values idxlist:', ((86345, 86345, 3197, 26), 1.0, (3197, 26)))
DEBUG:root:('change_values idxlist:', ((86399, 86399, 3199, 26), 1.0, (3199, 26)))


True

In [287]:
workset_before = workset_after.copy()

In [288]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==5]

,0,1,2,3,4
89548,89548,427,5,2,427
89549,89549,1654,5,-87.7005,86470
89550,89550,2086,5,-87.68878,86477
89551,89551,3436,5,-87.74887,86504
89552,89552,4099,5,2,4099
...,...,...,...,...,...
89633,89633,37645,5,-87.67212,87416
89634,89634,38401,5,-87.71057,87440
89635,89635,38941,5,-87.670747,87455
89636,89636,39589,5,-87.71622,87471


# cleanup minimum_nights and number_of_reviews

The columns "minimum_nights" and "number_of_reviews" should both be integer values. "minimum_nights" should be a value between 1 and the number of days in a year (365), while "number_of_reviews" should be a value between 0 and 999999.

To check if these columns meet the criteria, we have provided a "check_integer" function. Any values that do not meet the criteria should be cleaned to meet the criteria for analysis.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting "minimum_nights_flag" or "number_of_reviews_flag" to 1 or 2, respectively.

In [289]:
def check_integer(x,start=0,end=365):
    try:
        temp_x = int(x)
        return (start <= temp_x <= end)
    except:
        return False

In [290]:
def clean_minimum_nights(df):
    min_check_pd = df[df.minimum_nights_flag==0]
    min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,0,365))==False]
    wrong_min_ng= min_check_pd[["id","minimum_nights"]]
    ng_ind_ls=list(wrong_min_ng.index)

    for ng_id in ng_ind_ls:
        pre_ng=df.loc[ng_id,"minimum_nights"]
        
        try:
            if pre_ng==np.nan or math.isnan(pre_ng):
                # print("here:" +str(ng_id))
                df.loc[ng_id,"minimum_nights_flag"]=2
        except:
            try:
                revised=int(''.join(c for c in pre_ng if c.isdigit()))
                if check_integer(revised)==True:
                    df.loc[ng_id,"minimum_nights"]=revised
                else:
                    df.loc[ng_id,"minimum_nights_flag"]=1
        
            except:
                print(ng_id)
        
    return df

In [291]:
airbnb_pd = clean_minimum_nights(airbnb_pd)


In [292]:
def clean_min_nights_pls(nights_string):
    nights_string = re.sub("[^0-9.]", "", nights_string)
    nights = int(nights_string)
    return nights

In [293]:
airbnb_pd.loc[airbnb_pd['minimum_nights_flag']==1, 'minimum_nights'] = airbnb_pd.loc[airbnb_pd['minimum_nights_flag']==1, 'minimum_nights'].apply(lambda x: clean_min_nights_pls(x))
airbnb_pd.loc[airbnb_pd['minimum_nights_flag']==1, 'minimum_nights']
# nikolaus change minimum_nights_flag 1 to 0
airbnb_pd.loc[airbnb_pd['minimum_nights_flag']==1, 'minimum_nights_flag'] = 0


# Minimum nights checking

This query should return zero rows once you implement the cleaning process

In [294]:
min_check_pd = airbnb_pd[airbnb_pd.minimum_nights_flag==0]
min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,0,365))==False]
min_check_pd[["id","minimum_nights"]]

,id,minimum_nights
2168,42122343,1125
2173,42234877,1125


In [295]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [296]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                 0
name                               0
host_id                            0
host_name                          0
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                    34
number_of_reviews                  0
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
number_of_reviews_ltm              0
license                            0
id_flag                            0
host_id_flag                       0
neighbourhood_flag                 0
latitude_flag                      0
longitude_flag                     0
minimum_nights_flag               10
number_of_reviews_flag             0
last_review_flag                   0
room_type_flag                     0
d

In [297]:
dcm1.change_df(workset_before,workset_after,"clean_minimum_nights_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(41590, 87519, 1540, 10), (41806, 87529, 1548, 10), (42373, 87545, 1569, 10), (43507, 87569, 1611, 10), (43534, 87570, 1612, 10), (45410, 45410, 1681, 23), (46922, 46922, 1737, 23), (47746, 87688, 1768, 10), (48313, 87696, 1789, 10), (48853, 87703, 1809, 10), (49258, 87714, 1824, 10), (50554, 87747, 1872, 10), (51674, 51674, 1913, 23), (54172, 87838, 2006, 10), (56305, 87897, 2085, 10), (56332, 87902, 2086, 10), (57277, 87922, 2121, 10), (58546, 87957, 2168, 10), (58667, 58667, 2172, 23), (58681, 58681, 2173, 10), (58789, 87963, 2177, 10), (59356, 87973, 2198, 10), (61489, 88030, 2277, 10), (62596, 88063, 2318, 10), (62839, 88072, 2327, 10), (62893, 88075, 2329, 10), (64378, 88108, 2384, 10), (64783, 88119, 2399, 10), (65215, 88139, 2415, 10), (65512, 88148, 2426, 10), (67496, 67496, 2499, 23), (68684, 68684, 2543, 23), (70075, 88264, 2595, 10), (70210, 88266, 2600, 10), (70844, 70844, 2623, 23), (72181, 88319, 2673, 10), (72640, 88332, 2690, 10),

True

In [298]:
workset_before = workset_after.copy()

In [299]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==6]

,0,1,2,3,4
89638,89638,89,6,Entire home/apt,86428
89639,89639,170,6,Entire home/apt,86430
89640,89640,224,6,Entire home/apt,86431
89641,89641,359,6,Entire home/apt,86437
89642,89642,431,6,2,431
...,...,...,...,...,...
89831,89831,38510,6,Entire home/apt,87448
89832,89832,38780,6,Entire home/apt,87451
89833,89833,39104,6,Entire home/apt,87458
89834,89834,39455,6,Entire home/apt,87466


In [300]:
def clean_number_of_reviews(df):
    min_check_pd = df[df.number_of_reviews_flag==0]
    min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,0,999999))==False]
    wrong_min_nr=  min_check_pd[["id","number_of_reviews"]]
    nr_ind_ls=list(wrong_min_nr.index)

    for nr_id in nr_ind_ls:
        pre_nr=df.loc[nr_id,"number_of_reviews"]
        try:
            if pre_nr==np.nan or math.isnan(pre_nr):
                df.loc[nr_id,"number_of_reviews_flag"]=2
        except:
            try:
                revised=int(''.join(c for c in pre_nr if c.isdigit()))
                if check_integer(revised)==True:
                    df.loc[nr_id,"number_of_reviews"]=revised
                else:
                    df.loc[nr_id,"number_of_reviews_flag"]=1
        
            except:
                print(nr_id)
    return df

In [301]:
airbnb_pd = clean_number_of_reviews(airbnb_pd)

In [302]:
airbnb_pd[airbnb_pd['number_of_reviews_flag']==1]['number_of_reviews']

1486    47X5
1675    64r0
1676    64r0
Name: number_of_reviews, dtype: object

# Clean number of reviews checking

This query should return zero rows once you implement the cleaning process

In [303]:
min_check_pd = airbnb_pd[airbnb_pd.number_of_reviews_flag==0]
min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,1,999999))==False]
min_check_pd[["id","number_of_reviews"]]

,id,number_of_reviews
1529,5185790,0
1590,10634059,0
1598,12136030,0
1604,12444516,0
1605,12448898,0
...,...,...
3196,782628636832878491,0
3197,782643895516370805,0
3198,784994899201350568,0
3199,785423932330914663,0


In [304]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [305]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                 0
name                               0
host_id                            0
host_name                          0
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                 46
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
number_of_reviews_ltm              0
license                            0
id_flag                            0
host_id_flag                       0
neighbourhood_flag                 0
latitude_flag                      0
longitude_flag                     0
minimum_nights_flag                0
number_of_reviews_flag             9
last_review_flag                   0
room_type_flag                     0
d

In [306]:
dcm1.change_df(workset_before,workset_after,"clean_number_of_reviews_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(40146, 40146, 1486, 24), (40349, 87490, 1494, 11), (40619, 87496, 1504, 11), (41105, 87506, 1522, 11), (41240, 87512, 1527, 11), (41564, 87517, 1539, 11), (42212, 87541, 1563, 11), (42428, 87546, 1571, 11), (43022, 87557, 1593, 11), (43211, 87562, 1600, 11), (44102, 87586, 1633, 11), (44237, 87588, 1638, 11), (44453, 87592, 1646, 11), (45249, 45249, 1675, 24), (45276, 45276, 1676, 24), (45398, 87619, 1681, 11), (45425, 87622, 1682, 11), (45614, 87628, 1689, 11), (46248, 46248, 1712, 24), (46451, 87656, 1720, 11), (47234, 87678, 1749, 11), (49421, 87720, 1830, 11), (51203, 87760, 1896, 11), (51284, 87763, 1899, 11), (51405, 51405, 1903, 24), (52121, 87788, 1930, 11), (52931, 87807, 1960, 11), (52958, 87809, 1961, 11), (53876, 87831, 1995, 11), (54875, 87853, 2032, 11), (55118, 87856, 2041, 11), (55469, 87865, 2054, 11), (55496, 87867, 2055, 11), (56319, 56319, 2085, 24), (56360, 87904, 2087, 11), (56441, 87907, 2090, 11), (57075, 57075, 2113, 24),

True

In [307]:
workset_before = workset_after.copy()

In [308]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd[check_pd[2]==7]

,0,1,2,3,4
89836,89836,3277,7,32,86500
89837,89837,4100,7,2,4100
89838,89838,5734,7,32,86568
89839,89839,6220,7,120,86581
89840,89840,7300,7,32,86619
89841,89841,12214,7,32,86748
89842,89842,12862,7,32,86770
89843,89843,13321,7,500,13321
89844,89844,15346,7,32,86826
89845,89845,15697,7,32,86833


# cleanup last_review

The "last_review" column should be in the format of ISO-date (yyyy-mm-dd). We have provided a "check_date" function to verify the date format.

If a value is outside the date format or is null and you are unsure how to handle it, you can flag the row for deletion by setting the "last_review_flag" to 1 or 2.


In [309]:
from datetime import datetime
def check_date(x,fmt="%Y-%m-%d"):
    try:
        datetime.strptime(x,fmt)
        return True
    except:
        return False

In [310]:
import math
def clean_last_reviews(df):
    last_review_check_pd = df[df.last_review_flag==0]
    last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
    wrong_lr=last_review_check_pd[["id","last_review"]]
    lr_ind_ls=list(wrong_lr.index)
    
    for lr_id in lr_ind_ls:
        pre_lr=df.loc[lr_id,"last_review"]
        try:
            if pre_lr==np.nan or math.isnan(pre_lr) :
                df.loc[lr_id,"last_review_flag"]=2
        except:
            pre_lr=pre_lr.replace("/", " ")
            mon_ele=pre_lr.split()[0]
            date_ele=pre_lr.split()[1]
            year_ele=pre_lr.split()[2]
            revised_date="20"+year_ele+"-"+mon_ele+"-"+date_ele
            if check_date(revised_date)==True:
                df.loc[lr_id,"last_review"]=revised_date
            else:
                df.loc[lr_id,"last_review_flag"]=1
    return df

In [311]:
airbnb_pd = clean_last_reviews(airbnb_pd)

# Last Review checking

This query should return zero rows once you implement the cleaning process

In [312]:
last_review_check_pd = airbnb_pd[airbnb_pd.last_review_flag==0]
last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
last_review_check_pd[["id","last_review"]]

,id,last_review


In [313]:
workset_after = assign_to_original(workset_after,airbnb_pd)

In [314]:
(workset_after.fillna("")!=workset_before.fillna("")).sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                        72
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
id_flag                             0
host_id_flag                        0
neighbourhood_flag                  0
latitude_flag                       0
longitude_flag                      0
minimum_nights_flag                 0
number_of_reviews_flag              0
last_review_flag                  365
room_type_fl

In [315]:
dcm1.change_df(workset_before,workset_after,"clean_last_review_c2","c2")

DEBUG:root:change values
DEBUG:root:('change_values pd_index:',                            0                        1   \
0                (0, 0, 0, 0)             (1, 1, 0, 1)   
1              (27, 27, 1, 0)           (28, 28, 1, 1)   
2              (54, 54, 2, 0)           (55, 55, 2, 1)   
3              (81, 81, 3, 0)           (82, 82, 3, 1)   
4            (108, 108, 4, 0)         (109, 109, 4, 1)   
...                       ...                      ...   
3196  (86292, 86292, 3196, 0)  (86293, 86293, 3196, 1)   
3197  (86319, 86319, 3197, 0)  (86320, 86320, 3197, 1)   
3198  (86346, 86346, 3198, 0)  (86347, 86347, 3198, 1)   
3199  (86373, 86373, 3199, 0)  (86374, 86374, 3199, 1)   
3200  (86400, 86400, 3200, 0)  (86401, 86401, 3200, 1)   

                           2                        3   \
0                (2, 2, 0, 2)             (3, 3, 0, 3)   
1              (29, 29, 1, 2)           (30, 30, 1, 3)   
2              (56, 56, 2, 2)           (57, 57, 2, 3)   
3      

DEBUG:root:('change_values idxlist:', [(41133, 87507, 1523, 12), (41187, 87510, 1525, 12), (41308, 41308, 1529, 25), (41646, 87523, 1542, 12), (41916, 87532, 1552, 12), (42955, 42955, 1590, 25), (43171, 43171, 1598, 25), (43333, 43333, 1604, 25), (43360, 43360, 1605, 25), (43495, 43495, 1610, 25), (44400, 87591, 1644, 12), (44683, 44683, 1654, 25), (45115, 45115, 1670, 25), (45372, 87615, 1680, 12), (45412, 45412, 1681, 25), (45534, 87625, 1686, 12), (45588, 87627, 1688, 12), (45790, 45790, 1695, 25), (46249, 46249, 1712, 25), (46519, 46519, 1722, 25), (46843, 46843, 1734, 25), (47194, 47194, 1747, 25), (47343, 87681, 1753, 12), (47437, 47437, 1756, 25), (48099, 87694, 1781, 12), (48247, 48247, 1786, 25), (48423, 87697, 1793, 12), (48598, 48598, 1799, 25), (48679, 48679, 1802, 25), (49098, 87707, 1818, 12), (49368, 87716, 1828, 12), (49408, 49408, 1829, 25), (49962, 87732, 1850, 12), (50056, 50056, 1853, 25), (51325, 51325, 1900, 25), (51406, 51406, 1903, 25), (51433, 51433, 1904, 25),

DEBUG:root:('change_values idxlist:', ((41133, 87507, 1523, 12), '2022-08-08', (1523, 12)))
DEBUG:root:('change_values idxlist:', ((41187, 87510, 1525, 12), '2022-11-02', (1525, 12)))
DEBUG:root:('change_values idxlist:', ((41308, 41308, 1529, 25), 2.0, (1529, 25)))
DEBUG:root:('change_values idxlist:', ((41646, 87523, 1542, 12), '2022-06-15', (1542, 12)))
DEBUG:root:('change_values idxlist:', ((41916, 87532, 1552, 12), '2022-12-04', (1552, 12)))
DEBUG:root:('change_values idxlist:', ((42955, 42955, 1590, 25), 2.0, (1590, 25)))
DEBUG:root:('change_values idxlist:', ((43171, 43171, 1598, 25), 2.0, (1598, 25)))
DEBUG:root:('change_values idxlist:', ((43333, 43333, 1604, 25), 2.0, (1604, 25)))
DEBUG:root:('change_values idxlist:', ((43360, 43360, 1605, 25), 2.0, (1605, 25)))
DEBUG:root:('change_values idxlist:', ((43495, 43495, 1610, 25), 2.0, (1610, 25)))
DEBUG:root:('change_values idxlist:', ((44400, 87591, 1644, 12), '2022-11-12', (1644, 12)))
DEBUG:root:('change_values idxlist:', ((44

DEBUG:root:('change_values idxlist:', ((59506, 59506, 2203, 25), 2.0, (2203, 25)))
DEBUG:root:('change_values idxlist:', ((59641, 59641, 2208, 25), 2.0, (2208, 25)))
DEBUG:root:('change_values idxlist:', ((59722, 59722, 2211, 25), 2.0, (2211, 25)))
DEBUG:root:('change_values idxlist:', ((59803, 59803, 2214, 25), 2.0, (2214, 25)))
DEBUG:root:('change_values idxlist:', ((60249, 87997, 2231, 12), '2021-08-12', (2231, 12)))
DEBUG:root:('change_values idxlist:', ((60370, 60370, 2235, 25), 2.0, (2235, 25)))
DEBUG:root:('change_values idxlist:', ((60397, 60397, 2236, 25), 2.0, (2236, 25)))
DEBUG:root:('change_values idxlist:', ((60424, 60424, 2237, 25), 2.0, (2237, 25)))
DEBUG:root:('change_values idxlist:', ((60829, 60829, 2252, 25), 2.0, (2252, 25)))
DEBUG:root:('change_values idxlist:', ((61072, 61072, 2261, 25), 2.0, (2261, 25)))
DEBUG:root:('change_values idxlist:', ((61329, 88023, 2271, 12), '2022-02-10', (2271, 12)))
DEBUG:root:('change_values idxlist:', ((61383, 88026, 2273, 12), '202

DEBUG:root:('change_values idxlist:', ((70063, 70063, 2594, 25), 2.0, (2594, 25)))
DEBUG:root:('change_values idxlist:', ((70090, 70090, 2595, 25), 2.0, (2595, 25)))
DEBUG:root:('change_values idxlist:', ((70117, 70117, 2596, 25), 2.0, (2596, 25)))
DEBUG:root:('change_values idxlist:', ((70171, 70171, 2598, 25), 2.0, (2598, 25)))
DEBUG:root:('change_values idxlist:', ((70198, 70198, 2599, 25), 2.0, (2599, 25)))
DEBUG:root:('change_values idxlist:', ((70225, 70225, 2600, 25), 2.0, (2600, 25)))
DEBUG:root:('change_values idxlist:', ((70279, 70279, 2602, 25), 2.0, (2602, 25)))
DEBUG:root:('change_values idxlist:', ((70306, 70306, 2603, 25), 2.0, (2603, 25)))
DEBUG:root:('change_values idxlist:', ((70360, 70360, 2605, 25), 2.0, (2605, 25)))
DEBUG:root:('change_values idxlist:', ((70576, 70576, 2613, 25), 2.0, (2613, 25)))
DEBUG:root:('change_values idxlist:', ((70603, 70603, 2614, 25), 2.0, (2614, 25)))
DEBUG:root:('change_values idxlist:', ((70725, 88277, 2619, 12), '2022-11-19', (2619, 1

DEBUG:root:('change_values idxlist:', ((79027, 79027, 2926, 25), 2.0, (2926, 25)))
DEBUG:root:('change_values idxlist:', ((79297, 79297, 2936, 25), 2.0, (2936, 25)))
DEBUG:root:('change_values idxlist:', ((79459, 79459, 2942, 25), 2.0, (2942, 25)))
DEBUG:root:('change_values idxlist:', ((79513, 79513, 2944, 25), 2.0, (2944, 25)))
DEBUG:root:('change_values idxlist:', ((79581, 88520, 2947, 12), '2022-11-17', (2947, 12)))
DEBUG:root:('change_values idxlist:', ((79608, 88522, 2948, 12), '2022-12-13', (2948, 12)))
DEBUG:root:('change_values idxlist:', ((79675, 79675, 2950, 25), 2.0, (2950, 25)))
DEBUG:root:('change_values idxlist:', ((79689, 88525, 2951, 12), '2022-10-14', (2951, 12)))
DEBUG:root:('change_values idxlist:', ((79837, 79837, 2956, 25), 2.0, (2956, 25)))
DEBUG:root:('change_values idxlist:', ((79918, 79918, 2959, 25), 2.0, (2959, 25)))
DEBUG:root:('change_values idxlist:', ((79972, 79972, 2961, 25), 2.0, (2961, 25)))
DEBUG:root:('change_values idxlist:', ((80026, 80026, 2963, 

DEBUG:root:('change_values idxlist:', ((84724, 84724, 3137, 25), 2.0, (3137, 25)))
DEBUG:root:('change_values idxlist:', ((84751, 84751, 3138, 25), 2.0, (3138, 25)))
DEBUG:root:('change_values idxlist:', ((84805, 84805, 3140, 25), 2.0, (3140, 25)))
DEBUG:root:('change_values idxlist:', ((84832, 84832, 3141, 25), 2.0, (3141, 25)))
DEBUG:root:('change_values idxlist:', ((84940, 84940, 3145, 25), 2.0, (3145, 25)))
DEBUG:root:('change_values idxlist:', ((84967, 84967, 3146, 25), 2.0, (3146, 25)))
DEBUG:root:('change_values idxlist:', ((84994, 84994, 3147, 25), 2.0, (3147, 25)))
DEBUG:root:('change_values idxlist:', ((85075, 85075, 3150, 25), 2.0, (3150, 25)))
DEBUG:root:('change_values idxlist:', ((85102, 85102, 3151, 25), 2.0, (3151, 25)))
DEBUG:root:('change_values idxlist:', ((85129, 85129, 3152, 25), 2.0, (3152, 25)))
DEBUG:root:('change_values idxlist:', ((85156, 85156, 3153, 25), 2.0, (3153, 25)))
DEBUG:root:('change_values idxlist:', ((85237, 85237, 3156, 25), 2.0, (3156, 25)))
DEBU

True

In [316]:
workset_before = workset_after.copy()

In [317]:
check_pd = pd.DataFrame(dcm1.cell_values)
check_pd.groupby(2).count()

,0,1,3,4
2,,,,
-1,86427,86427,81484,86427
0,2291,2291,2291,2291
1,70,70,70,70
2,354,354,354,354
3,322,322,322,322
4,84,84,84,84
5,90,90,90,90
6,198,198,198,198
7,28,28,28,28


In [318]:
pd.DataFrame(dcm1.column_position)

,0,1,2,3,4,5
0,0,0,-1,id,-1,-1
1,1,1,-1,name,0,-1
2,2,2,-1,host_id,1,-1
3,3,3,-1,host_name,2,-1
4,4,4,-1,neighbourhood_group,3,-1
5,5,5,-1,neighbourhood,4,-1
6,6,6,-1,latitude,5,-1
7,7,7,-1,longitude,6,-1
8,8,8,-1,room_type,7,-1
9,9,9,-1,price,8,-1


In [319]:
dcm1.state_detail[0] = (0, {'op': 'initial'}, -1, -1, None)

In [320]:
pd.DataFrame(dcm1.state_detail)

,0,1,2,3,4
0,0,{'op': 'initial'},-1,-1,NaN
1,0,perturbed_dataset,0,-1,0.0
2,1,clean_duplicate_id_c1,1,0,1.0
3,2,clean_inconsistent_host_id_c1,2,1,1.0
4,3,clean_neighbourhood_c1,3,2,1.0
5,4,clean_latitude_c1,4,3,1.0
6,5,clean_longitude_c1,5,4,1.0
7,6,clean_room_type_c1,6,5,1.0
8,7,clean_minimum_nights_c1,7,6,1.0
9,8,clean_number_of_reviews_c1,8,7,1.0


In [321]:
cv = pd.DataFrame(dcm1.cell_values)
cv[cv[2]==9]

,0,1,2,3,4
89911,89911,66,9,2022-11-28,86427
89912,89912,79,9,0,79
89913,89913,430,9,2,430
89914,89914,741,9,2022-11-27,86448
89915,89915,754,9,0,754
...,...,...,...,...,...
90320,90320,40012,9,2,40012
90321,90321,40039,9,2,40039
90322,90322,40066,9,2,40066
90323,90323,40093,9,2,40093


In [322]:
pd.DataFrame(dcm1.state)

,0,1
0,-1,-2
1,0,-1
2,1,0
3,2,1
4,3,2
5,4,3
6,5,4
7,6,5
8,7,6
9,8,7


In [323]:
pd.DataFrame(dcm1.cell_values).merge(pd.DataFrame(dcm1.cell),left_on=1,right_on=0,suffixes=["_x","_y"]).\
merge(pd.DataFrame(dcm1.column_position),left_on="1_y",right_on=1,suffixes=["_xx","_z"]).\
merge(pd.DataFrame(dcm1.state_detail),left_on="2_x",right_on=0,suffixes=["_zz","_k"])

,1_xx,0_x,1_x,2_x,3_xx,4_xx,0_y,1_y,2_y,0_zz,1_z,2_zz,3_z,4_z,5,0_k,1,2_k,3,4
0,4699,86536,4699,0,null,4699,4699,1,174,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
1,4699,86536,4699,0,null,4699,4699,1,174,1,1,-1,name,0,-1,0,perturbed_dataset,0,-1,0.0
2,5563,86558,5563,0,null,5563,5563,1,206,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
3,5563,86558,5563,0,null,5563,5563,1,206,1,1,-1,name,0,-1,0,perturbed_dataset,0,-1,0.0
4,6913,86607,6913,0,null,6913,6913,1,256,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797,85994,91397,85994,15,1.0,85994,85994,26,3184,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7798,86264,91398,86264,15,1.0,86264,86264,26,3194,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7799,86291,91399,86291,15,1.0,86291,86291,26,3195,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7800,86345,91400,86345,15,1.0,86345,86345,26,3197,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0


In [324]:
check_pd = pd.DataFrame(dcm1.cell_values).merge(pd.DataFrame(dcm1.cell),left_on=1,right_on=0,suffixes=["_x","_y"]). \
    merge(pd.DataFrame(dcm1.column_position),left_on="1_y",right_on=1,suffixes=["_xx","_z"]). \
    merge(pd.DataFrame(dcm1.state_detail),left_on="2_x",right_on=0,suffixes=["_zz","_k"]) #.drop_duplicates()


In [325]:
check_pd

,1_xx,0_x,1_x,2_x,3_xx,4_xx,0_y,1_y,2_y,0_zz,1_z,2_zz,3_z,4_z,5,0_k,1,2_k,3,4
0,4699,86536,4699,0,null,4699,4699,1,174,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
1,4699,86536,4699,0,null,4699,4699,1,174,1,1,-1,name,0,-1,0,perturbed_dataset,0,-1,0.0
2,5563,86558,5563,0,null,5563,5563,1,206,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
3,5563,86558,5563,0,null,5563,5563,1,206,1,1,-1,name,0,-1,0,perturbed_dataset,0,-1,0.0
4,6913,86607,6913,0,null,6913,6913,1,256,1,1,-1,name,0,-1,0,{'op': 'initial'},-1,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797,85994,91397,85994,15,1.0,85994,85994,26,3184,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7798,86264,91398,86264,15,1.0,86264,86264,26,3194,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7799,86291,91399,86291,15,1.0,86291,86291,26,3195,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
7800,86345,91400,86345,15,1.0,86345,86345,26,3197,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0


In [326]:
cv

,0,1,2,3,4
0,0,0,-1,25879,-1
1,1,1,-1,2/1 One Block to Fullerton L Red Line Deck & ...,-1
2,2,2,-1,101521,-1
3,3,3,-1,Red,-1
4,4,4,-1,NaN,-1
...,...,...,...,...,...
91933,91933,86317,18,2.0,86317
91934,91934,86344,18,2.0,86344
91935,91935,86371,18,2.0,86371
91936,91936,86398,18,2.0,86398


In [327]:
cv = pd.DataFrame(dcm1.cell_values)
cellv = pd.DataFrame(dcm1.cell)
cv[cv[2]==0].merge(cellv[cellv[1]==6],left_on=1,right_on=0)
#test_cv = cv[cv[2]==4].merge(cv[cv[2]==-1],left_on=1,right_on=1)
#test_cv[test_cv["3_x"].apply(lambda x:float(x))==test_cv["3_y"].apply(lambda x:float(x))]

,1,0_x,1_x,2_x,3,4,0_y,1_y,2_y
0,3624,86509,3624,0,E41.9A2748,3624,3624,6,134
1,5082,86548,5082,0,4L1.9320u3,5082,5082,6,188
2,5163,86551,5163,0,41.9vB3123,5163,5163,6,191
3,6216,86580,6216,0,E41.91 334,6216,6216,6,230
4,6243,86582,6243,0,4N1C.69097,6243,6243,6,231
...,...,...,...,...,...,...,...,...,...
189,85785,88695,85785,0,418167,85785,85785,6,3177
190,85893,88699,85893,0,418945286,85893,85893,6,3181
191,85974,88708,85974,0,4190362,85974,85974,6,3184
192,86190,88711,86190,0,4190271,86190,86190,6,3192


In [328]:
cellv[cellv[0]==3624]

,0,1,2
3624,3624,6,134


In [329]:
test_cv = cv[cv[2]==0]
test_cv[test_cv[1]==3624]
test_cv[test_cv[0]==89348]
cv[cv[1]==3624]

,0,1,2,3,4
3624,3624,3624,-1,41.92748,-1
86509,86509,3624,0,E41.9A2748,3624
89464,89464,3624,4,41.92748,86509


In [330]:
cellv[cellv[1]==6].shape

(3201, 3)

In [331]:
pd.DataFrame(dcm1.column_position)
cellv[cellv[1]==6]

,0,1,2
6,6,6,0
33,33,6,1
60,60,6,2
87,87,6,3
114,114,6,4
...,...,...,...
86298,86298,6,3196
86325,86325,6,3197
86352,86352,6,3198
86379,86379,6,3199


In [332]:
cv = pd.DataFrame(dcm1.cell_values)
test_cv = cv[cv[2]==4].merge(cv[cv[2]==-1],left_on=1,right_on=1)
test_cv[test_cv["3_x"].apply(lambda x:float(x))==test_cv["3_y"].apply(lambda x:float(x))]

,0_x,1,2_x,3_x,4_x,0_y,2_y,3_y,4_y
0,89464,3624,4,41.92748,86509,3624,-1,41.92748,-1
1,89465,5082,4,41.93203,86548,5082,-1,41.93203,-1
2,89466,5163,4,41.93123,86551,5163,-1,41.93123,-1
3,89467,6216,4,41.91334,86580,6216,-1,41.91334,-1
4,89468,6243,4,41.69097,86582,6243,-1,41.69097,-1
...,...,...,...,...,...,...,...,...,...
79,89543,37428,4,41.955908,87410,37428,-1,41.9559085,-1
80,89544,37455,4,42.00246,87411,37455,-1,42.00246,-1
81,89545,37779,4,41.881121,87418,37779,-1,41.8811213,-1
82,89546,38049,4,41.84067,87430,38049,-1,41.84067,-1


In [333]:
test_cv[test_cv["3_x"].apply(lambda x:float(x))!=test_cv["3_y"].apply(lambda x:float(x))]

,0_x,1,2_x,3_x,4_x,0_y,2_y,3_y,4_y
32,89496,14493,4,2,14493,14493,-1,0,-1


In [334]:
check_pd[check_pd[4].isin([0,1,2])].drop_duplicates().groupby([4,1,"3_z"]).count()[["1_xx"]] #.to_csv("test.csv")

1_xx
4   1                             3_z                         
0.0 perturbed_dataset             host_name                154
                                  last_review              145
                                  latitude                 194
                                  longitude                185
                                  minimum_nights            70
                                  name                      18
                                  neighbourhood            695
                                  number_of_reviews        102
                                  price                     14
                                  reviews_per_month        296
                                  room_type                418
1.0 clean_duplicate_id_c1         id_flag                   70
    clean_inconsistent_host_id_c1 host_id_flag             354
    clean_last_review_c1          last_review               64
                                  last_review_flag         350
    clean_latitude_c1             latitude                  83
                                  latitude_flag              1
    clean_longitude_c1            longitude                 80
                                  longitude_flag            10
    clean_minimum_nights_c1       minimum_nights            24
                                  minimum_nights_flag        4
    clean_neighbourhood_c1        neighbourhood            316
                                  neighbourhood_flag         6
    clean_number_of_reviews_c1    number_of_reviews         41
                                  number_of_reviews_flag     6
    clean_room_type_c1            room_type                189
                                  room_type_flag             9
2.0 clean_duplicate_id_c2         id_flag                   69
    clean_inconsistent_host_id_c2 host_id_flag             214
    clean_last_review_c2          last_review               72
                                  last_review_flag         365
    clean_latitude_c2             latitude_flag            106
    clean_longitude_c2            longitude_flag            95
    clean_minimum_nights_c2       minimum_nights            34
                                  minimum_nights_flag       10
    clean_neighbourhood_c2        neighbourhood            367
                                  neighbourhood_flag         6
    clean_number_of_reviews_c2    number_of_reviews         46
                                  number_of_reviews_flag     9
    clean_room_type_c2            room_type_flag           220

In [301]:
check_pd

,1_xx,0_x,1_x,2_x,3_xx,4_xx,0_y,1_y,2_y,0_zz,1_z,2_zz,3_z,4_z,5,0_k,1,2_k,3,4
0,0,0,0,-1,25879,-1,0,0,0,0,0,-1,id,-1,-1,0,{'op': 'initial'},-1,-1,NaN
1,27,27,27,-1,37738,-1,27,0,1,0,0,-1,id,-1,-1,0,{'op': 'initial'},-1,-1,NaN
2,54,54,54,-1,189821,-1,54,0,2,0,0,-1,id,-1,-1,0,{'op': 'initial'},-1,-1,NaN
3,81,81,81,-1,207218,-1,81,0,3,0,0,-1,id,-1,-1,0,{'op': 'initial'},-1,-1,NaN
4,108,108,108,-1,220333,-1,108,0,4,0,0,-1,id,-1,-1,0,{'op': 'initial'},-1,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93928,85994,91217,85994,14,1.0,85994,85994,26,3184,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
93929,86264,91218,86264,14,1.0,86264,86264,26,3194,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
93930,86291,91219,86291,14,1.0,86291,86291,26,3195,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0
93931,86345,91220,86345,14,1.0,86345,86345,26,3197,26,26,-1,room_type_flag,25,-1,15,clean_room_type_c2,15,14,2.0


In [134]:
dcm1.col_id

27

In [ ]:
# write datalog facts
with open("dcm_2.pl","w") as file:
    for x in set(dcm1.column):
        file.write('column("{}").\n'.format('","'.join([str(y) for y in x])))
    for x in set(dcm1.row):
        file.write('row("{}").\n'.format('","'.join([str(y) for y in x])))   
    for x in dcm1.cell:
        file.write('cell("{}").\n'.format('","'.join([str(y) for y in x])))
    for x in dcm1.cell_values:
        file.write('cell_values("{}").\n'.format('","'.join([str(y) for y in x])))
    for x in dcm1.col_dependency:
        file.write('col_dependency("{}").\n'.format('","'.join([str(y) for y in x])))
    for x in dcm1.value_derived_from:
        file.write('value_derived_from("{}").\n'.format('","'.join([str(y) for y in x])))
    for x in dcm1.column_position:
        file.write('column_position("{}").\n'.format('","'.join([str(y) for y in x])))

In [ ]:
#save dcmx in pickle
import pickle
#with open("dcm_collab.pickle","wb") as file:
#    pickle.dump(dcm1,file)